In [1]:
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt

import pickle

import seaborn as sns

%matplotlib qt
mpl.rcParams['lines.linewidth'] = 0.91
plt.style.use('seaborn-v0_8-whitegrid')

## Load GGIR output 
(From script: `/Volumes/Untitled/rehab/GGIR/GGIR_LG-miar.Rmd`)

In [2]:
import pyreadr

In [3]:
diary_SPT = {    
    "158": [pd.Timestamp('2024-02-28 23:00:00'), pd.Timestamp('2024-02-29 07:15:00')], # 158 OK
    "633": [pd.Timestamp('2024-03-07 00:05:00'), pd.Timestamp('2024-03-07 06:36:00')], # 633 OK
    "906": [pd.Timestamp('2024-03-07 00:30:00'), pd.Timestamp('2024-03-07 07:30:00')], # 906 OK
    "958": [pd.Timestamp('2024-03-13 22:00:00'), pd.Timestamp('2024-03-14 06:00:00')], # 958 OK
    "127": [pd.Timestamp('2024-03-13 23:15:00'), pd.Timestamp('2024-03-14 06:50:00')], # 127 OK
    "098": [pd.Timestamp('2024-03-16 02:01:00'), pd.Timestamp('2024-03-16 09:50:00')], # 098 OK
    "547": [pd.Timestamp('2024-03-16 01:04:00'), pd.Timestamp('2024-03-16 07:40:00')], # 547 OK
    "815": [pd.Timestamp('2024-03-20 23:00:00'), pd.Timestamp('2024-03-21 06:25:00')], # 815 OK
    "914": [pd.Timestamp('2024-03-20 21:50:00'), pd.Timestamp('2024-03-21 05:50:00')], # 914 OK
    "971": [pd.Timestamp('2024-03-20 23:50:00'), pd.Timestamp('2024-03-21 07:50:00')], # 971 OK
    "279": [pd.Timestamp('2024-03-28 00:10:00'), pd.Timestamp('2024-03-28 07:27:00')], # 279 OK
    "965": [pd.Timestamp('2024-03-28 01:25:00'), pd.Timestamp('2024-03-28 09:20:00')], # 965 OK
}

diary_TIB = {
    "158": [pd.Timestamp('2024-02-28 22:15:00'), pd.Timestamp('2024-02-29 07:45:00')], # 158 OK
    "633": [pd.Timestamp('2024-03-06 23:39:00'), pd.Timestamp('2024-03-07 08:00:00')], # 633 OK 
    "906": [pd.Timestamp('2024-03-07 00:15:00'), pd.Timestamp('2024-03-07 07:35:00')], # 906 OK
    "958": [pd.Timestamp('2024-03-13 21:30:00'), pd.Timestamp('2024-03-14 06:30:00')], # 958 OK
    "127": [pd.Timestamp('2024-03-13 22:00:00'), pd.Timestamp('2024-03-14 07:10:00')], # 127 OK 
    "098": [pd.Timestamp('2024-03-16 01:49:00'), pd.Timestamp('2024-03-16 09:52:00')], # 098 OK 
    "547": [pd.Timestamp('2024-03-16 00:26:00'), pd.Timestamp('2024-03-16 08:20:00')], # 547 OK 
    "815": [pd.Timestamp('2024-03-20 22:00:00'), pd.Timestamp('2024-03-21 07:30:00')], # 815 OK 
    "914": [pd.Timestamp('2024-03-20 21:30:00'), pd.Timestamp('2024-03-21 06:20:00')], # 914 OK 
    "971": [pd.Timestamp('2024-03-20 23:30:00'), pd.Timestamp('2024-03-21 08:08:00')], # 971 OK 
    "279": [pd.Timestamp('2024-03-28 00:04:00'), pd.Timestamp('2024-03-28 07:41:00')], # 279 OK
    "965": [pd.Timestamp('2024-03-28 01:22:00'), pd.Timestamp('2024-03-28 09:22:00')], # 965 OK
}

In [4]:
from functions.bursts import characterize_bursts

In [70]:
part2_outputFolder = "/Volumes/Untitled/rehab/GGIR/GGIR_output_lw_TIB/output_lw_data/meta/ms2.out/"
part3_outputFolder = "/Volumes/Untitled/rehab/GGIR/GGIR_output_lw_TIB/output_lw_data/meta/ms3.out/"
subjects = ["158", "098", "633", "279", "906", "547", "971", "958", "815"]

SIB_GGIR = {sub: pyreadr.read_r(part3_outputFolder + "LW_" + sub + ".CWA.RData")['sib.cla.sum'][["sib.onset.time", "sib.end.time"]] for sub in subjects}

bursts_lw = {sub: 0 for sub in subjects}
bursts_rw = {sub: 0 for sub in subjects}
bursts_ll = {sub: 0 for sub in subjects}
bursts_rl = {sub: 0 for sub in subjects}
bursts_trunk = {sub: 0 for sub in subjects}
bursts_all_limbs = {sub: 0 for sub in subjects}
bursts_all_limbs_new = {sub: 0 for sub in subjects}
SIB = {sub: 0 for sub in subjects}

bursts_df = pd.DataFrame()

for i, sub in enumerate(subjects):
    SIB_GGIR[sub]["sib.onset.time"] = pd.to_datetime(SIB_GGIR[sub]["sib.onset.time"].values).tz_localize(None)
    SIB_GGIR[sub]["sib.end.time"] = pd.to_datetime(SIB_GGIR[sub]["sib.end.time"].values).tz_localize(None)
    # end time - onset time
    SIB_GGIR[sub]["sib.duration"] = SIB_GGIR[sub]["sib.end.time"] - SIB_GGIR[sub]["sib.onset.time"]
    # onset time - previous end time
    # print(min(SIB_all[sub].dropna()["awake.duration"]))
    # SIB[sub]['sib.onset.time'] += pd.Timedelta('5s')
    # SIB[sub]['sib.end.time'] += pd.Timedelta('5s')
    with open(f'/Volumes/Untitled/rehab/data/{sub}/bursts_TIB.pkl', 'rb') as f:
        bursts = pickle.load(f)
    # bursts_lw[sub] = bursts["lw"]
    # bursts_rw[sub] = bursts["rw"]
    # bursts_ll[sub] = bursts["ll"]
    # bursts_rl[sub] = bursts["rl"]
    # bursts_trunk[sub] = bursts["trunk"]
    # bursts_all_limbs[sub] = bursts["all_limbs"]

    df_merged_intervals = characterize_bursts(bursts)
    spt_start = diary_SPT[sub][0] - pd.Timedelta('10 min')
    spt_end = diary_TIB[sub][1] + pd.Timedelta('5 min')

    SIB[sub] = SIB_GGIR[sub][(SIB_GGIR[sub]["sib.onset.time"] >= spt_start) & (SIB_GGIR[sub]["sib.end.time"] <= spt_end)].reset_index(drop=True)
    SIB[sub] = SIB_GGIR[sub][(SIB_GGIR[sub]["sib.onset.time"] >= spt_start) & (SIB_GGIR[sub]["sib.end.time"] <= spt_end)].reset_index(drop=True)

    # Take df_merged_intervals between spt_start and spt_end
    df_merged_intervals = df_merged_intervals[(df_merged_intervals["Start"] >= spt_start) & (df_merged_intervals["End"] <= spt_end)].reset_index(drop=True) 

    SIB[sub]["awake.duration"] = SIB[sub]["sib.onset.time"].shift(-1) - SIB[sub]["sib.end.time"]
    SIB[sub]["sub_ID"] = sub

    df_merged_intervals["SIB"] = 0
    for i, row in SIB[sub].iterrows():
        df_merged_intervals.loc[(df_merged_intervals["Start"] >= row["sib.onset.time"] + pd.Timedelta("5s")) & (df_merged_intervals["End"] <= row["sib.end.time"] - pd.Timedelta("5s")), "SIB"] = 1

    df_merged_intervals["sub_ID"] = sub

    start_sleep = diary_SPT[sub][0] - pd.Timedelta('10 min')
    end_sleep = diary_SPT[sub][1] + pd.Timedelta('10 min')

    df_merged_intervals = df_merged_intervals.loc[(df_merged_intervals["Start"] >= start_sleep) & (df_merged_intervals["End"] <= end_sleep)]

    bursts_df = pd.concat([bursts_df, df_merged_intervals])

    # Angle-z
    # metashort = pd.read_csv(part2_outputFolder + "/metashortLW_" + sub + ".CWA.RData.csv", index_col=1)
    # metashort.index = pd.to_datetime(metashort.index).tz_localize(None)
    # anglez = metashort.drop(columns=["Unnamed: 0", "ENMO"])

    # bursts_lw[sub]["sub_ID"] = sub
    # bursts_rw[sub]["sub_ID"] = sub
    # bursts_ll[sub]["sub_ID"] = sub
    # bursts_rl[sub]["sub_ID"] = sub
    # bursts_trunk[sub]["sub_ID"] = sub
    # bursts_all_limbs[sub]["sub_ID"] = sub

    # Find bursts that overlap with SIB
    # bursts_lw[sub]["sib"] = 0
    # for i, sib in SIB[sub].iterrows():
    #     for j, burst in bursts_lw[sub].iterrows():
    #         if burst["start"] > sib["sib.onset.time"] + pd.Timedelta('5s') and burst["end"] < sib["sib.end.time"] - pd.Timedelta('5s'):
    #             bursts_lw[sub].at[j, "sib"] = 1
    # bursts_rw[sub]["sib"] = 0
    # for i, sib in SIB[sub].iterrows():
    #     for j, burst in bursts_rw[sub].iterrows():
    #         if burst["start"] > sib["sib.onset.time"] + pd.Timedelta('5s') and burst["end"] < sib["sib.end.time"] - pd.Timedelta('5s'):
    #             bursts_rw[sub].at[j, "sib"] = 1
    # bursts_ll[sub]["sib"] = 0
    # for i, sib in SIB[sub].iterrows():
    #     for j, burst in bursts_ll[sub].iterrows():
    #         if burst["start"] > sib["sib.onset.time"] + pd.Timedelta('5s') and burst["end"] < sib["sib.end.time"] - pd.Timedelta('5s'):
    #             bursts_ll[sub].at[j, "sib"] = 1
    # bursts_rl[sub]["sib"] = 0
    # for i, sib in SIB[sub].iterrows():
    #     for j, burst in bursts_rl[sub].iterrows():
    #         if burst["start"] > sib["sib.onset.time"] + pd.Timedelta('5s') and burst["end"] < sib["sib.end.time"] - pd.Timedelta('5s'):
    #             bursts_rl[sub].at[j, "sib"] = 1
    # bursts_trunk[sub]["sib"] = 0
    # for i, sib in SIB[sub].iterrows():
    #     for j, burst in bursts_trunk[sub].iterrows():
    #         if burst["start"] > sib["sib.onset.time"] + pd.Timedelta('5s') and burst["end"] < sib["sib.end.time"] - pd.Timedelta('5s'):
    #             bursts_trunk[sub].at[j, "sib"] = 1
    # bursts_all_limbs[sub]["sib"] = 0
    # for i, sib in SIB[sub].iterrows():
    #     for j, burst in bursts_all_limbs[sub].iterrows():
    #         if burst["start"] > sib["sib.onset.time"] + pd.Timedelta('5s') and burst["end"] < sib["sib.end.time"] - pd.Timedelta('5s'):
    #             bursts_all_limbs[sub].at[j, "sib"] = 1
bursts_df = bursts_df.reset_index(drop=True)

## EDA - Single limbs but burst may happen together

SIB contains the SIB for all subjects (start time, end time and duration). Obviously, it contains also the AWAKE periods (total of n-1 events). 

In [ ]:
# One DataFrame for all subjects
SIB_all = pd.concat(SIB.values(), ignore_index=True)
SIB_all["awake.duration.seconds"] = SIB_all["awake.duration"].dt.total_seconds()
SIB_all["sib.duration.seconds"] = SIB_all["sib.duration"].dt.total_seconds()
SIB_all["sib.duration.minutes"] = SIB_all["sib.duration"].dt.total_seconds() / 60
SIB_all["awake.duration.minutes"] = SIB_all["awake.duration"].dt.total_seconds() / 60

SIB and bursts overlap - 1 subject

In [ ]:
plt.figure()
plt.plot(anglez)
for i, sib in SIB[sub].iterrows():
    plt.axvspan(sib["sib.onset.time"], sib["sib.end.time"], color='r', alpha=0.5)
for j, burst in bursts_lw[sub].iterrows():
    plt.axvspan(burst["start"], burst["end"], color='b', alpha=0.5)

Duration of awake periods between SIB

In [ ]:
# Plot the number of awake periods between SIB
sns.set_context("talk")

plt.figure()
sns.stripplot(data=SIB_all, x="sub_ID", y="awake.duration.minutes", jitter=0.2, alpha = 0.5, color = "blue")
sns.boxplot(data=SIB_all, x="sub_ID", y="awake.duration.minutes", showfliers=False, color = "y")
plt.ylabel("Awake duration (minutes)")

plt.title("Awake periods between SIB during Time In Bed")

Text(0.5, 1.0, 'Awake periods between SIB during Time In Bed')

Duration of SIB

In [8]:
# Plot the number of SIB
sns.set_context("talk")

plt.figure()
sns.stripplot(data=SIB_all, x="sub_ID", y="sib.duration.minutes", jitter=0.2, alpha = 0.5, color = "blue")
sns.boxplot(data=SIB_all, x="sub_ID", y="sib.duration.minutes", showfliers=False, color = "pink")
plt.ylabel("SIB duration (minutes)")

plt.title("SIB during Time In Bed")

Text(0.5, 1.0, 'SIB during Time In Bed')

Get a burst DataFrame for all subjects for each limb

In [9]:
bursts_lw_allSub = pd.concat(bursts_lw.values(), ignore_index=True)
bursts_lw_allSub["duration_seconds"] = bursts_lw_allSub["duration"].dt.total_seconds()
bursts_rw_allSub = pd.concat(bursts_rw.values(), ignore_index=True)
bursts_rw_allSub["duration_seconds"] = bursts_rw_allSub["duration"].dt.total_seconds()
bursts_ll_allSub = pd.concat(bursts_ll.values(), ignore_index=True)
bursts_ll_allSub["duration_seconds"] = bursts_ll_allSub["duration"].dt.total_seconds()
bursts_rl_allSub = pd.concat(bursts_rl.values(), ignore_index=True)
bursts_rl_allSub["duration_seconds"] = bursts_rl_allSub["duration"].dt.total_seconds()
bursts_trunk_allSub = pd.concat(bursts_trunk.values(), ignore_index=True)
bursts_trunk_allSub["duration_seconds"] = bursts_trunk_allSub["duration"].dt.total_seconds()

Number of bursts happening during sleep (SIB) and during wake

In [10]:
burst_count_lw = bursts_lw_allSub["sib"].groupby(bursts_lw_allSub["sub_ID"]).value_counts().unstack()
burst_count_rw = bursts_rw_allSub["sib"].groupby(bursts_rw_allSub["sub_ID"]).value_counts().unstack()
burst_count_ll = bursts_ll_allSub["sib"].groupby(bursts_ll_allSub["sub_ID"]).value_counts().unstack()
burst_count_rl = bursts_rl_allSub["sib"].groupby(bursts_rl_allSub["sub_ID"]).value_counts().unstack()
burst_count_trunk = bursts_trunk_allSub["sib"].groupby(bursts_trunk_allSub["sub_ID"]).value_counts().unstack()

sns.set_context("talk")

# Bar plot
plt.figure(figsize = (15, 12))
plt.subplot(3, 2, 1)
burst_count_lw.plot(kind='bar', color = ['blue', 'red'], ax = plt.gca())
plt.legend([])
plt.ylabel("N of bursts")
plt.xticks(rotation = 19)
plt.subplot(3, 2, 2)
burst_count_rw.plot(kind='bar', color = ['blue', 'red'], ax = plt.gca())
plt.legend([])
plt.ylabel("N of bursts")
plt.xticks(rotation = 19)
plt.subplot(3, 2, 3)
burst_count_ll.plot(kind='bar', color = ['blue', 'red'], ax = plt.gca())
plt.legend([])
plt.ylabel("N of bursts")
plt.xticks(rotation = 19)
plt.subplot(3, 2, 4)
burst_count_rl.plot(kind='bar', color = ['blue', 'red'], ax = plt.gca())
plt.legend([])
plt.ylabel("N of bursts")
plt.xticks(rotation = 19)
plt.subplot(3, 2, 5)
burst_count_trunk.plot(kind='bar', color = ['blue', 'red'], ax = plt.gca())
plt.ylabel("N of bursts")
plt.xticks(rotation = 19)

from matplotlib.patches import Patch

custom_patches = [Patch(facecolor="b"),
                Patch(facecolor="r")]

plt.legend(custom_patches, ["Awake", "Asleep"], loc = 'best', fontsize = 19, frameon = True, fancybox = True, shadow = True)
plt.ylabel("Number of bursts")
plt.xticks(rotation = 19)
plt.suptitle("Number of bursts during SIB and Awake Periods")
plt.tight_layout()


Posture changes (at least 30°) happening during sleep and during wake

In [13]:
# Posture changes

bursts_lw_posture_changes = bursts_lw_allSub[bursts_lw_allSub["posture_change"] > 30]

bursts_posture_changes_counts = bursts_lw_posture_changes["sib"].groupby(bursts_lw_allSub["sub_ID"]).value_counts().unstack()

plt.figure()
bursts_posture_changes_counts.plot(kind='bar', color = ['blue', 'red'])
from matplotlib.patches import Patch

custom_patches = [Patch(facecolor="b"),
                Patch(facecolor="r")]

plt.legend(custom_patches, ["Awake", "Asleep"], loc = 'best', fontsize = 19, frameon = True, fancybox = True, shadow = True)
plt.ylabel("Number of posture changes")
plt.xticks(rotation = 19)
plt.title("Posture changes during SIB and Awake Periods")

Text(0.5, 1.0, 'Posture changes during SIB and Awake Periods')

Burst entity (AUC) during sleep and during wake 

In [11]:
import seaborn as sns
sns.set_context('talk')

# Set up the matplotlib figure
plt.figure(figsize=(19, 12))

# Boxplot for AUC
plt.subplot(3, 2, 1)
ax1 = sns.boxplot(x='sib', y='AUC', data=bursts_lw_allSub, hue = "sub_ID", showfliers=False)
plt.title('LW')
plt.ylabel('Movement Intensity (AUC)')
plt.xlabel(None)
ax1.legend_.remove()
plt.xticks([0, 1], ['Awake', 'Asleep'])

plt.subplot(3, 2, 2)
ax2 = sns.boxplot(x='sib', y='AUC', data=bursts_rw_allSub, hue = "sub_ID", showfliers=False)
plt.title('RW')
plt.ylabel(None)
plt.xlabel(None)
ax2.legend_.remove()
plt.xticks([0, 1], ['Awake', 'Asleep'])

plt.subplot(3, 2, 3)
ax3 = sns.boxplot(x='sib', y='AUC', data=bursts_ll_allSub, hue = "sub_ID", showfliers=False)
plt.title('LL')
plt.ylabel(None)
plt.xlabel(None)
ax3.legend_.remove()
plt.xticks([0, 1], ['Awake', 'Asleep'])

plt.subplot(3, 2, 4)
ax4 = sns.boxplot(x='sib', y='AUC', data=bursts_rl_allSub, hue = "sub_ID", showfliers=False)
plt.title('RL')
plt.ylabel(None)
plt.xlabel(None)
ax4.legend_.remove()
plt.xticks([0, 1], ['Awake', 'Asleep'])
handles, labels = [], []

for handle, label in zip(*ax4.get_legend_handles_labels()):
    if label not in labels:
        handles.append(handle)
        labels.append(label)

plt.legend(handles, labels, loc='lower right', frameon=True, fancybox=True, shadow=True, bbox_to_anchor=(0.8, -1.1), ncol=3, fontsize = 18)

plt.subplot(3, 2, 5)
ax5 = sns.boxplot(x='sib', y='AUC', data=bursts_trunk_allSub, hue = "sub_ID", showfliers=False)
plt.title('Trunk')
plt.ylabel(None)
plt.xlabel(None)
ax5.legend_.remove()
plt.xticks([0, 1], ['Awake', 'Asleep'])

plt.show()

## EDA - Bursts happening in all limbs together

Merge the bursts of all subjects into one DataFrame

In [5]:
bursts_all_limbs_allSub = pd.concat(bursts_all_limbs.values(), ignore_index=True)

Number of bursts happening during sleep (SIB) and during wake

In [54]:
burst_count = bursts_all_limbs_allSub["sib"].groupby(bursts_all_limbs_allSub["sub_ID"]).value_counts().unstack()
sns.set_context("talk")

# Bar plot
plt.figure(figsize = (15, 12))
burst_count.plot(kind='bar', color = ['blue', 'red'], ax = plt.gca())
plt.legend([])
plt.ylabel("N of bursts")
plt.xticks(rotation = 19)

from matplotlib.patches import Patch

custom_patches = [Patch(facecolor="b"),
                Patch(facecolor="r")]

plt.legend(custom_patches, ["Awake", "Asleep"], loc = 'best', fontsize = 24, frameon = True, fancybox = True, shadow = True)
plt.ylabel("Number of bursts")
plt.xticks(rotation = 19)
plt.suptitle("Number of bursts during SIB and Awake Periods")
plt.tight_layout()

Burst entity (AUC) during sleep and during wake 

In [40]:
import seaborn as sns
sns.set_context('talk')

# Set up the matplotlib figure
plt.figure(figsize=(19, 12))

# Boxplot for AUC
ax1 = sns.boxplot(x='sib', y='AUC', data=bursts_all_limbs_allSub, hue = "sub_ID", showfliers=False)
plt.title('Movement Entity during sleep and wake')
plt.ylabel('Burst Entity (AUC)')
plt.xlabel(None)
ax1.legend_.remove()
plt.xticks([0, 1], ['Awake', 'Asleep'])

handles, labels = [], []

for handle, label in zip(*ax4.get_legend_handles_labels()):
    if label not in labels:
        handles.append(handle)
        labels.append(label)

plt.legend(handles, labels, loc='upper right', frameon=True, fancybox=True, shadow=True, ncol=3, fontsize = 21)

Percentiles of AUC

In [113]:
burst_describe = bursts_all_limbs_allSub.groupby(["sub_ID", "sib"])["AUC"].describe(percentiles = [0.9]).unstack()
burst_describe

count             mean                 std                 min  \
sib         0      1         0         1         0         1         0   
sub_ID                                                                   
098      44.0   20.0  6.653740  0.554661  7.152739  0.848130  0.087940   
158      63.0   44.0  5.404391  0.821867  8.144159  1.343849  0.072168   
279      18.0   19.0  7.570457  0.521787  9.849087  0.470343  0.196278   
547      58.0   27.0  6.906781  0.394636  7.425048  0.554090  0.066033   
633     138.0   67.0  3.676741  0.538104  4.664902  1.051418  0.033980   
815     143.0  110.0  4.272575  0.653986  9.421159  1.212396  0.028514   
906      39.0   46.0  3.971676  0.928337  5.091755  1.849077  0.049147   
958      53.0   32.0  4.028301  0.853181  3.692125  1.447341  0.189213   
971      47.0   30.0  3.183710  0.342831  3.814307  0.479453  0.050280   

                       50%                  90%                  max            
sib            1         0         1          0         1          0         1  
sub_ID                                                                          
098     0.036440  4.312882  0.219282  15.466636  1.699520  32.016583  3.147857  
158     0.020699  2.458432  0.337410  12.912712  2.124245  41.595520  7.972749  
279     0.022541  3.636635  0.410731  19.048691  1.270420  36.922621  1.393919  
547     0.016537  4.881188  0.143086  17.244877  1.287616  35.248337  1.977761  
633     0.016485  1.679356  0.142984  10.269015  1.647083  23.853609  5.364481  
815     0.022363  2.218856  0.207835   8.016730  1.829797  90.555946  8.442378  
906     0.020909  2.010235  0.154564   9.270132  2.312497  26.160641  8.204787  
958     0.024885  2.471979  0.153863   8.968217  2.740480  16.743444  6.185938  
971     0.024210  1.755934  0.160333   8.352138  0.633358  14.629342  2.113501

In [115]:
# Are there awake burts with less or equal entity than the maximum asleep burst?

AUC_sleep90 = burst_describe["90%"][1]

In [129]:
burst_awake_low_AUC = {sub: 0 for sub in subjects}
burst_sleep_low_AUC = {sub: 0 for sub in subjects}
for i, sub in enumerate(subjects):
    bursts_awake = bursts_all_limbs[sub][bursts_all_limbs[sub]["sib"] == 0]
    bursts_sleep = bursts_all_limbs[sub][bursts_all_limbs[sub]["sib"] == 1]

    # Find bursts with less or equal entity than the maximum asleep burst
    burst_awake_low_AUC[sub] = bursts_awake[bursts_awake["AUC"] <= AUC_sleep90[sub]]
    burst_sleep_low_AUC[sub] = bursts_sleep#bursts_sleep[bursts_sleep["AUC"] <= AUC_sleep90[sub]]

# One DataFrame for all subjects
bursts_awake_low_AUC_all = pd.concat(burst_awake_low_AUC.values(), ignore_index=True)
bursts_sleep_low_AUC_all = pd.concat(burst_sleep_low_AUC.values(), ignore_index=True)

In [130]:
# merge the two DataFrames
bursts_low_AUC_all = pd.concat([bursts_awake_low_AUC_all, bursts_sleep_low_AUC_all], ignore_index=True)
bursts_low_AUC_all["sib"] = bursts_low_AUC_all["sib"].replace({0: "Awake", 1: "Asleep"})
bursts_low_AUC_all

,start,end,limbs_involved,AUC,posture_change,sub_ID,sib
0,2024-02-28 22:19:07.308890104,2024-02-28 22:19:15.331500053,"{ll, rw, rl, lw, trunk}",0.249701,NaN,158,Awake
1,2024-02-28 22:19:43.688889980,2024-02-28 22:20:18.178889990,"{ll, rw, rl, lw, trunk}",0.588274,NaN,158,Awake
2,2024-02-28 22:20:39.318890095,2024-02-28 22:21:01.828890085,"{ll, rw, rl, lw, trunk}",0.429421,NaN,158,Awake
3,2024-02-28 22:36:40.808890104,2024-02-28 22:36:53.901499987,"{ll, rw, rl, lw, trunk}",0.211791,NaN,158,Awake
4,2024-02-28 22:36:57.844870090,2024-02-28 22:37:23.088890076,"{ll, rw, rl, lw, trunk}",1.566868,NaN,158,Awake
...,...,...,...,...,...,...,...
649,2024-03-21 07:17:09.782916069,2024-03-21 07:17:13.544266224,"{ll, rw, rl, lw, trunk}",0.068493,NaN,815,Asleep
650,2024-03-21 07:19:17.504266499,2024-03-21 07:19:23.362916231,"{ll, rw, rl, lw, trunk}",0.064362,NaN,815,Asleep
651,2024-03-21 07:19:45.114266157,2024-03-21 07:19:51.562916517,"{ll, rw, rl, lw, trunk}",0.187400,NaN,815,Asleep
652,2024-03-21 07:20:13.392916203,2024-03-21 07:20:20.444266319,"{ll, rw, rl, lw, trunk}",0.203871,NaN,815,Asleep


In [122]:
plt.figure(figsize=(15, 6))
plt.subplot(1, 2, 1)
sns.boxplot(x='sub_ID', y='AUC', data=bursts_awake_low_AUC_all, showfliers=False)
sns.stripplot(x='sub_ID', y='AUC', data=bursts_awake_low_AUC_all, jitter=0.2, alpha = 0.5, color = "red")
plt.subplot(1, 2, 1)
sns.boxplot(x='sub_ID', y='AUC', data=bursts_sleep_low_AUC_all, showfliers=False,  whis=[5, 90])
sns.stripplot(x='sub_ID', y='AUC', data=bursts_sleep_low_AUC_all, jitter=0.2, alpha = 0.5, color = "red")


<Axes: xlabel='sub_ID', ylabel='AUC'>

In [135]:
plt.figure(figsize=(15, 6))
sns.boxplot(x='sub_ID', y='AUC', data=bursts_low_AUC_all, hue = "sib")
plt.ylabel("Movement intensity (AUC)")

Text(0, 0.5, 'Movement intensity (AUC)')

In [141]:
bursts_awake_low_AUC_all.value_counts("sub_ID").sort_index()

sub_ID
098    12
158    30
279     7
547    13
633    69
815    67
906    21
958    28
971    12
Name: count, dtype: int64

In [142]:
bursts_sleep_low_AUC_all.value_counts("sub_ID").sort_index()


sub_ID
098     20
158     44
279     19
547     27
633     67
815    110
906     46
958     32
971     30
Name: count, dtype: int64

## HR response during wakefullness

Is there a dose-response relationship? (based on the three percentiles)

In [71]:
bursts_df.shape

(1857, 7)

In [6]:
bursts_all_limbs = bursts_df[bursts_df["Limbs"] == set(("LL", "LW", "RL", "RW", "T"))].reset_index(drop=True)

bursts_LW = bursts_df[bursts_df["Limbs"] == {"LW"}].reset_index(drop=True)
bursts_RW = bursts_df[bursts_df["Limbs"] == {"RW"}].reset_index(drop=True)
bursts_LL = bursts_df[bursts_df["Limbs"] == {"LL"}].reset_index(drop=True)
bursts_RL = bursts_df[bursts_df["Limbs"] == {"RL"}].reset_index(drop=True)

bursts_wrists = bursts_df[(bursts_df["Limbs"] == {"RW"}) | (bursts_df["Limbs"] == {"LW"})].reset_index(drop=True)
bursts_ankles = bursts_df[(bursts_df["Limbs"] == {"RL"}) | (bursts_df["Limbs"] == {"LL"})].reset_index(drop=True)

bursts_LL_RL_T = bursts_df[bursts_df["Limbs"] == set(("LL", "RL", "T"))].reset_index(drop=True)

bursts_LL_RL_or_LL_RL_T = bursts_df[(bursts_df["Limbs"] == {"LL", "RL"}) | (bursts_df["Limbs"] == {"LL", "RL", "T"})].reset_index(drop=True)

bursts_single_limbs =  bursts_df[(bursts_df["Limbs"] == {"LW"}) | (bursts_df["Limbs"] == {"RW"}) | (bursts_df["Limbs"] == {"RL"}) | (bursts_df["Limbs"] == {"LL"})].reset_index(drop=True)

In [7]:
import neurokit2 as nk

from functions.HR_response import detect_HR_change_from_RR, coherent_avg
from functions.bursts import filter_bursts

In [60]:
offsets = {"158": 2, "098": 2, "633": 2, "279": 3, "906": 2, "547": 2, "971": 2, "958": 2, "815": 2}

#### 5 LIMBS ####
hr_responses_spt = {}
hr_responses_high = {}
hr_responses_med = {}
hr_responses_low = {}

hr_responses_awake = {}
hr_responses_awake_high = {}
hr_responses_awake_med = {}
hr_responses_awake_low = {}

hr_responses_sleep = {}
hr_responses_sleep_high = {}
hr_responses_sleep_med = {}
hr_responses_sleep_low = {}

#### LW ####
hr_responses_LW = {}
hr_responses_awake_LW = {}
hr_responses_sleep_LW = {}

#### RW ####
hr_responses_RW = {}
hr_responses_awake_RW = {}
hr_responses_sleep_RW = {}

#### LL ####
hr_responses_LL = {}
hr_responses_awake_LL = {}
hr_responses_sleep_LL = {}

#### RL ####
hr_responses_RL = {}
hr_responses_awake_RL = {}
hr_responses_sleep_RL = {}

#### BOTH WRISTS ####
hr_responses_wrists = {}
hr_responses_awake_wrists = {}
hr_responses_sleep_wrists = {}

#### BOTH ANKLES ####
hr_responses_ankles = {}
hr_responses_awake_ankles = {}
hr_responses_sleep_ankles = {}

#### LL, RL, T ####
hr_responses_LL_RL_T = {}
hr_responses_awake_LL_RL_T = {}
hr_responses_sleep_LL_RL_T = {}

#### LL, RL or LL, RL, T ####
hr_responses_LL_RL_or_LL_RL_T = {}
hr_responses_awake_LL_RL_or_LL_RL_T = {}
hr_responses_sleep_LL_RL_or_LL_RL_T = {}

#### SINGLE LIMBS ####
hr_responses_single_limbs = {}
hr_responses_awake_single_limbs = {}
hr_responses_sleep_single_limbs = {}

# HR peaks
hr_peaks_spt = {}
hr_peaks_sleep_high = {}
hr_peaks_sleep_med = {}
hr_peaks_sleep_low = {}
hr_peaks_singleLimbs = {}

# HR latencies
hr_latencies_spt = {}
hr_latencies_singleLimbs = {}


for i, sub in enumerate(subjects):

    print(sub)

    off = offsets[sub]

    ecg_df = pd.read_pickle(f'/Volumes/Untitled/rehab/data/{sub}/polar_processed/ecg.pkl')
    start_sleep, end_sleep = diary_SPT[sub]

    ecg_df = ecg_df.loc[start_sleep:end_sleep]
    ecg_filtered = nk.ecg_clean(ecg_df.values, sampling_rate=130)

    # Extract peaks
    _, results = nk.ecg_peaks(ecg_filtered, sampling_rate=130, method = 'neurokit')
    rpeaks = results["ECG_R_Peaks"]
    _, rpeaks_corrected = nk.signal_fixpeaks(rpeaks, sampling_rate=130, iterative=True, method="Kubios")

    t_rpeaks = ecg_df.index.to_series().values[rpeaks]
    t_rpeaks_corrected = ecg_df.index.to_series().values[rpeaks_corrected]
    rr = np.diff(t_rpeaks).astype('timedelta64[ns]').astype('float64') / 1000000000
    rr_corrected = np.diff(t_rpeaks_corrected).astype('timedelta64[ns]').astype('float64') / 1000000000
    hr_ecg = 60/rr
    hr_ecg_corrected = 60/rr_corrected
    hr_df = pd.Series(hr_ecg_corrected, index = t_rpeaks_corrected[1:]).resample("1 s").mean()#.rolling('10s', min_periods=1, center=True).mean()
    hr_df = hr_df.interpolate(method = 'cubic')
    hr_df_noncorrected = pd.Series(hr_ecg, index = t_rpeaks[1:]).resample("1 s").mean()
    hr_df_noncorrected = hr_df_noncorrected.interpolate(method = 'linear')

    artifacts_ecg = pd.read_csv(f'/Volumes/Untitled/rehab/data/{sub}/polar_processed/artifacts_ecg.csv')
    artifacts_ecg['Start'] = pd.to_datetime(artifacts_ecg['Start']).apply(lambda x: x.replace(tzinfo=None))
    artifacts_ecg['End'] = pd.to_datetime(artifacts_ecg['End']).apply(lambda x: x.replace(tzinfo=None))

    for i in range(len(artifacts_ecg)):
        hr_df.loc[artifacts_ecg["Start"].iloc[i]:artifacts_ecg["End"].iloc[i]] = np.nan

    hr_df.interpolate(method = 'cubic', inplace = True)

    for i in range(len(artifacts_ecg)):
        if artifacts_ecg["End"].iloc[i] - artifacts_ecg["Start"].iloc[i] > pd.Timedelta("20 s"):
            hr_df.loc[artifacts_ecg["Start"].iloc[i]:artifacts_ecg["End"].iloc[i]] = np.nan

    #### PLOT #### 
    # plt.figure(figsize=(15, 9))
    # plt.plot(hr_df, label = "HR corrected")
    # for i, artifact in artifacts_ecg.iterrows():
    #     plt.axvspan(artifact["Start"], artifact["End"], color = "red", alpha = 0.3)
    # plt.title(f"HR corrected {sub}")

    # SIB_sub = SIB[sub]
    # for i, sib in SIB_sub.iterrows():
    #     plt.axvspan(sib["sib.onset.time"], sib["sib.end.time"], color = "green", alpha = 0.3)

    # plt.vlines(bursts_all_limbs[sub][bursts_all_limbs[sub]["sib"] == 0]["start"], ymin = 0, ymax = 200, color = "blue", alpha = 0.3)

    ##### ALL LIMBS MOVING #####

    bursts_awake = bursts_all_limbs.loc[(bursts_all_limbs["sub_ID"] == sub) & (bursts_all_limbs["SIB"] == 0)].reset_index(drop=True)
    bursts_sleep = bursts_all_limbs.loc[(bursts_all_limbs["sub_ID"] == sub) & (bursts_all_limbs["SIB"] == 1)].reset_index(drop=True)
    # Cut according to start_sleep and end_sleep
    bursts_all = bursts_all_limbs.loc[(bursts_all_limbs["Start"] >= start_sleep) & (bursts_all_limbs["End"] <= end_sleep)].reset_index(drop=True) # I dont select the current sub, the time takes care of it already
    bursts_awake = bursts_awake.loc[(bursts_awake["Start"] >= start_sleep) & (bursts_awake["End"] <= end_sleep)] 
    bursts_sleep = bursts_sleep.loc[(bursts_sleep["Start"] >= start_sleep) & (bursts_sleep["End"] <= end_sleep)]

    # plt.figure(figsize=(15, 9)) 
    # for i in range(bursts_all.shape[0]):
    #     plt.axvspan(bursts_all["Start"].iloc[i], bursts_all["End"].iloc[i], color = "blue", alpha = 0.3)
    #     plt.plot(hr_df.loc[bursts_all["Start"].iloc[i]:bursts_all["End"].iloc[i]], color = "red")

    HR_bursts, HR_peak, latency_HR_peak, AUC, _ = detect_HR_change_from_RR(filter_bursts(bursts_all), hr_df, offsets[sub]+1, plot = False)
    HR_bursts_awake, HR_peak_awake, _, AUC_awake, _ = detect_HR_change_from_RR(filter_bursts(bursts_awake), hr_df, offsets[sub]+1, plot = False)
    HR_bursts_sleep, HR_peak_sleep, _, AUC_sleep, _ = detect_HR_change_from_RR(filter_bursts(bursts_sleep), hr_df, offsets[sub]+1, plot = False)

    # Small vs medium vs large movements (regarless of sleep or wake)
    AUC_33 = np.percentile(AUC, 33)
    AUC_66 = np.percentile(AUC, 66)

    HR_bursts_low = [HR_bursts[i] for i in range(len(HR_bursts)) if AUC[i] <= AUC_33]
    HR_bursts_med = [HR_bursts[i] for i in range(len(HR_bursts)) if (AUC[i] > AUC_33) and (AUC[i] <= AUC_66)]
    HR_bursts_high = [HR_bursts[i] for i in range(len(HR_bursts)) if AUC[i] > AUC_66]

    HR_change, _ = coherent_avg(HR_bursts)
    HR_change_low, _ = coherent_avg(HR_bursts_low)
    HR_change_med, _ = coherent_avg(HR_bursts_med)
    HR_change_high, _ = coherent_avg(HR_bursts_high)

    # Small vs medium vs large movements during wake
    AUC_33_awake = np.percentile(AUC_awake, 33)
    AUC_66_awake = np.percentile(AUC_awake, 66)

    HR_bursts_awake_low = [HR_bursts_awake[i] for i in range(len(HR_bursts_awake)) if AUC_awake[i] <= AUC_33_awake]
    HR_bursts_awake_med = [HR_bursts_awake[i] for i in range(len(HR_bursts_awake)) if (AUC_awake[i] > AUC_33_awake) and (AUC_awake[i] <= AUC_66_awake)]
    HR_bursts_awake_high = [HR_bursts_awake[i] for i in range(len(HR_bursts_awake)) if AUC_awake[i] > AUC_66_awake]

    HR_change_awake, _ = coherent_avg(HR_bursts_awake)
    HR_change_awake_low, _ = coherent_avg(HR_bursts_awake_low)
    HR_change_awake_med, _ = coherent_avg(HR_bursts_awake_med)
    HR_change_awake_high, _ = coherent_avg(HR_bursts_awake_high)

    # Small vs medium vs large movements during sleep
    AUC_33_sleep = np.percentile(AUC_sleep, 33)
    AUC_66_sleep = np.percentile(AUC_sleep, 66)

    HR_bursts_sleep_low = [HR_bursts_sleep[i] for i in range(len(HR_bursts_sleep)) if AUC_sleep[i] <= AUC_33_sleep]
    HR_bursts_sleep_med = [HR_bursts_sleep[i] for i in range(len(HR_bursts_sleep)) if (AUC_sleep[i] > AUC_33_sleep) and (AUC_sleep[i] <= AUC_66_sleep)]
    HR_bursts_sleep_high = [HR_bursts_sleep[i] for i in range(len(HR_bursts_sleep)) if AUC_sleep[i] > AUC_66_sleep]

    HR_peak_sleep_low = [HR_peak_sleep[i] for i in range(len(HR_peak_sleep)) if AUC_sleep[i] <= AUC_33_sleep]
    HR_peak_sleep_med = [HR_peak_sleep[i] for i in range(len(HR_peak_sleep)) if (AUC_sleep[i] > AUC_33_sleep) and (AUC_sleep[i] <= AUC_66_sleep)]
    HR_peak_sleep_high = [HR_peak_sleep[i] for i in range(len(HR_peak_sleep)) if AUC_sleep[i] > AUC_66_sleep]

    HR_change_sleep, _ = coherent_avg(HR_bursts_sleep)
    HR_change_sleep_low, _ = coherent_avg(HR_bursts_sleep_low)
    HR_change_sleep_med, _ = coherent_avg(HR_bursts_sleep_med)
    HR_change_sleep_high, _ = coherent_avg(HR_bursts_sleep_high)

    #### LW ####
    bursts_awake_lw = bursts_LW.loc[(bursts_LW["sub_ID"] == sub) & (bursts_LW["SIB"] == 0)].reset_index(drop=True)
    bursts_sleep_lw = bursts_LW.loc[(bursts_LW["sub_ID"] == sub) & (bursts_LW["SIB"] == 1)].reset_index(drop=True)
    # Cut according to start_sleep and end_sleep
    bursts_lw = bursts_LW.loc[(bursts_LW["Start"] >= start_sleep) & (bursts_LW["End"] <= end_sleep)].reset_index(drop=True) # I dont select the current sub, the time takes care of it already
    bursts_awake_lw = bursts_awake_lw.loc[(bursts_awake_lw["Start"] >= start_sleep) & (bursts_awake_lw["End"] <= end_sleep)]
    bursts_sleep_lw = bursts_sleep_lw.loc[(bursts_sleep_lw["Start"] >= start_sleep) & (bursts_sleep_lw["End"] <= end_sleep)]

    HR_bursts_lw, _, _, AUC_lw, _ = detect_HR_change_from_RR(filter_bursts(bursts_lw), hr_df, offsets[sub]+1, plot = False)
    HR_bursts_awake_lw, _, _, AUC_awake_lw, _ = detect_HR_change_from_RR(filter_bursts(bursts_awake_lw), hr_df, offsets[sub]+1, plot = False)
    HR_bursts_sleep_lw, _, _, AUC_sleep_lw, _ = detect_HR_change_from_RR(filter_bursts(bursts_sleep_lw), hr_df, offsets[sub]+1, plot = False)
    HR_change_lw, _ = coherent_avg(HR_bursts_lw)
    HR_change_awake_lw, _ = coherent_avg(HR_bursts_awake_lw)
    HR_change_sleep_lw, _ = coherent_avg(HR_bursts_sleep_lw)

    #### RW ####
    bursts_awake_rw = bursts_RW.loc[(bursts_RW["sub_ID"] == sub) & (bursts_RW["SIB"] == 0)].reset_index(drop=True)
    bursts_sleep_rw = bursts_RW.loc[(bursts_RW["sub_ID"] == sub) & (bursts_RW["SIB"] == 1)].reset_index(drop=True)
    # Cut according to start_sleep and end_sleep
    bursts_rw = bursts_RW.loc[(bursts_RW["Start"] >= start_sleep) & (bursts_RW["End"] <= end_sleep)].reset_index(drop=True) # I dont select the current sub, the time takes care of it already
    bursts_awake_rw = bursts_awake_rw.loc[(bursts_awake_rw["Start"] >= start_sleep) & (bursts_awake_rw["End"] <= end_sleep)]
    bursts_sleep_rw = bursts_sleep_rw.loc[(bursts_sleep_rw["Start"] >= start_sleep) & (bursts_sleep_rw["End"] <= end_sleep)]

    HR_bursts_rw, _, _, AUC_rw, _ = detect_HR_change_from_RR(filter_bursts(bursts_rw), hr_df, offsets[sub]+1, plot = False)
    HR_bursts_awake_rw, _, _, AUC_awake_rw, _ = detect_HR_change_from_RR(filter_bursts(bursts_awake_rw), hr_df, offsets[sub]+1, plot = False)
    HR_bursts_sleep_rw, _, _, AUC_sleep_rw, _ = detect_HR_change_from_RR(filter_bursts(bursts_sleep_rw), hr_df, offsets[sub]+1, plot = False)
    HR_change_rw, _ = coherent_avg(HR_bursts_rw)
    HR_change_awake_rw, _ = coherent_avg(HR_bursts_awake_rw)
    HR_change_sleep_rw, _ = coherent_avg(HR_bursts_sleep_rw)

    #### LL ####
    bursts_awake_ll = bursts_LL.loc[(bursts_LL["sub_ID"] == sub) & (bursts_LL["SIB"] == 0)].reset_index(drop=True)
    bursts_sleep_ll = bursts_LL.loc[(bursts_LL["sub_ID"] == sub) & (bursts_LL["SIB"] == 1)].reset_index(drop=True)
    # Cut according to start_sleep and end_sleep
    bursts_ll = bursts_LL.loc[(bursts_LL["Start"] >= start_sleep) & (bursts_LL["End"] <= end_sleep)].reset_index(drop=True) # I dont select the current sub, the time takes care of it already
    bursts_awake_ll = bursts_awake_ll.loc[(bursts_awake_ll["Start"] >= start_sleep) & (bursts_awake_ll["End"] <= end_sleep)]
    bursts_sleep_ll = bursts_sleep_ll.loc[(bursts_sleep_ll["Start"] >= start_sleep) & (bursts_sleep_ll["End"] <= end_sleep)]

    HR_bursts_ll, _, _, AUC_ll, _ = detect_HR_change_from_RR(filter_bursts(bursts_ll), hr_df, offsets[sub]+1, plot = False)
    HR_bursts_awake_ll, _, _, AUC_awake_ll, _ = detect_HR_change_from_RR(filter_bursts(bursts_awake_ll), hr_df, offsets[sub]+1, plot = False)
    HR_bursts_sleep_ll, _, _, AUC_sleep_ll, _ = detect_HR_change_from_RR(filter_bursts(bursts_sleep_ll), hr_df, offsets[sub]+1, plot = False)
    HR_change_ll, _ = coherent_avg(HR_bursts_ll)
    HR_change_awake_ll, _ = coherent_avg(HR_bursts_awake_ll)
    HR_change_sleep_ll, _ = coherent_avg(HR_bursts_sleep_ll)

    #### RL ####
    bursts_awake_rl = bursts_RL.loc[(bursts_RL["sub_ID"] == sub) & (bursts_RL["SIB"] == 0)].reset_index(drop=True)
    bursts_sleep_rl = bursts_RL.loc[(bursts_RL["sub_ID"] == sub) & (bursts_RL["SIB"] == 1)].reset_index(drop=True)
    # Cut according to start_sleep and end_sleep
    bursts_rl = bursts_RL.loc[(bursts_RL["Start"] >= start_sleep) & (bursts_RL["End"] <= end_sleep)].reset_index(drop=True) # I dont select the current sub, the time takes care of it already
    bursts_awake_rl = bursts_awake_rl.loc[(bursts_awake_rl["Start"] >= start_sleep) & (bursts_awake_rl["End"] <= end_sleep)]
    bursts_sleep_rl = bursts_sleep_rl.loc[(bursts_sleep_rl["Start"] >= start_sleep) & (bursts_sleep_rl["End"] <= end_sleep)]
    
    HR_bursts_rl, _, _, AUC_rl, _ = detect_HR_change_from_RR(filter_bursts(bursts_rl), hr_df, offsets[sub]+1, plot = False)
    HR_bursts_awake_rl, _, _, AUC_awake_rl, _ = detect_HR_change_from_RR(filter_bursts(bursts_awake_rl), hr_df, offsets[sub]+1, plot = False)
    HR_bursts_sleep_rl, _, _, AUC_sleep_rl, _ = detect_HR_change_from_RR(filter_bursts(bursts_sleep_rl), hr_df, offsets[sub]+1, plot = False)
    HR_change_rl, _ = coherent_avg(HR_bursts_rl)
    HR_change_awake_rl, _ = coherent_avg(HR_bursts_awake_rl)
    HR_change_sleep_rl, _ = coherent_avg(HR_bursts_sleep_rl)

    #### BOTH WRISTS ####
    bursts_awake_wrists = bursts_wrists.loc[(bursts_wrists["sub_ID"] == sub) & (bursts_wrists["SIB"] == 0)].reset_index(drop=True)
    bursts_sleep_wrists = bursts_wrists.loc[(bursts_wrists["sub_ID"] == sub) & (bursts_wrists["SIB"] == 1)].reset_index(drop=True)
    # Cut according to start_sleep and end_sleep
    bursts_Wrists = bursts_wrists.loc[(bursts_wrists["Start"] >= start_sleep) & (bursts_wrists["End"] <= end_sleep)].reset_index(drop=True) # I dont select the current sub, the time takes care of it already
    bursts_awake_wrists = bursts_awake_wrists.loc[(bursts_awake_wrists["Start"] >= start_sleep) & (bursts_awake_wrists["End"] <= end_sleep)]
    bursts_sleep_wrists = bursts_sleep_wrists.loc[(bursts_sleep_wrists["Start"] >= start_sleep) & (bursts_sleep_wrists["End"] <= end_sleep)]

    HR_bursts_wrists, _, _, AUC_wrists, _ = detect_HR_change_from_RR(filter_bursts(bursts_Wrists), hr_df, offsets[sub]+1, plot = False)
    HR_bursts_awake_wrists, _, _, AUC_awake_wrists, _ = detect_HR_change_from_RR(filter_bursts(bursts_awake_wrists), hr_df, offsets[sub]+1, plot = False)
    HR_bursts_sleep_wrists, _, _, AUC_sleep_wrists, _ = detect_HR_change_from_RR(filter_bursts(bursts_sleep_wrists), hr_df, offsets[sub]+1, plot = False)
    HR_change_wrists, _ = coherent_avg(HR_bursts_wrists)
    HR_change_awake_wrists, _ = coherent_avg(HR_bursts_awake_wrists)
    HR_change_sleep_wrists, _ = coherent_avg(HR_bursts_sleep_wrists)

    #### BOTH ANKLES ####
    bursts_awake_ankles = bursts_ankles.loc[(bursts_ankles["sub_ID"] == sub) & (bursts_ankles["SIB"] == 0)].reset_index(drop=True)
    bursts_sleep_ankles = bursts_ankles.loc[(bursts_ankles["sub_ID"] == sub) & (bursts_ankles["SIB"] == 1)].reset_index(drop=True)
    # Cut according to start_sleep and end_sleep
    bursts_Ankles = bursts_ankles.loc[(bursts_ankles["Start"] >= start_sleep) & (bursts_ankles["End"] <= end_sleep)].reset_index(drop=True) # I dont select the current sub, the time takes care of it already
    bursts_awake_ankles = bursts_awake_ankles.loc[(bursts_awake_ankles["Start"] >= start_sleep) & (bursts_awake_ankles["End"] <= end_sleep)]
    bursts_sleep_ankles = bursts_sleep_ankles.loc[(bursts_sleep_ankles["Start"] >= start_sleep) & (bursts_sleep_ankles["End"] <= end_sleep)]

    HR_bursts_ankles, _, _, AUC_ankles, _ = detect_HR_change_from_RR(filter_bursts(bursts_Ankles), hr_df, offsets[sub]+1, plot = False)
    HR_bursts_awake_ankles, _, _, AUC_awake_ankles, _ = detect_HR_change_from_RR(filter_bursts(bursts_awake_ankles), hr_df, offsets[sub]+1, plot = False)
    HR_bursts_sleep_ankles, _, _, AUC_sleep_ankles, _ = detect_HR_change_from_RR(filter_bursts(bursts_sleep_ankles), hr_df, offsets[sub]+1, plot = False)
    HR_change_ankles, _ = coherent_avg(HR_bursts_ankles)
    HR_change_awake_ankles, _ = coherent_avg(HR_bursts_awake_ankles)
    HR_change_sleep_ankles, _ = coherent_avg(HR_bursts_sleep_ankles)

    #### LL, RL, T ####
    bursts_awake_LL_RL_T = bursts_LL_RL_T.loc[(bursts_LL_RL_T["sub_ID"] == sub) & (bursts_LL_RL_T["SIB"] == 0)].reset_index(drop=True)
    bursts_sleep_LL_RL_T = bursts_LL_RL_T.loc[(bursts_LL_RL_T["sub_ID"] == sub) & (bursts_LL_RL_T["SIB"] == 1)].reset_index(drop=True)
    # Cut according to start_sleep and end_sleep
    bursts_ll_rl_t = bursts_LL_RL_T.loc[(bursts_LL_RL_T["Start"] >= start_sleep) & (bursts_LL_RL_T["End"] <= end_sleep)].reset_index(drop=True) # I dont select the current sub, the time takes care of it already
    bursts_awake_LL_RL_T = bursts_awake_LL_RL_T.loc[(bursts_awake_LL_RL_T["Start"] >= start_sleep) & (bursts_awake_LL_RL_T["End"] <= end_sleep)]
    bursts_sleep_LL_RL_T = bursts_sleep_LL_RL_T.loc[(bursts_sleep_LL_RL_T["Start"] >= start_sleep) & (bursts_sleep_LL_RL_T["End"] <= end_sleep)]

    HR_bursts_LL_RL_T, _, _, AUC_LL_RL_T, _ = detect_HR_change_from_RR(filter_bursts(bursts_ll_rl_t), hr_df, offsets[sub]+1, plot = False)
    HR_bursts_awake_LL_RL_T, _, _, AUC_awake_LL_RL_T, _ = detect_HR_change_from_RR(filter_bursts(bursts_awake_LL_RL_T), hr_df, offsets[sub]+1, plot = False)
    HR_bursts_sleep_LL_RL_T, _, _, AUC_sleep_LL_RL_T, _ = detect_HR_change_from_RR(filter_bursts(bursts_sleep_LL_RL_T), hr_df, offsets[sub]+1, plot = False)
    HR_change_LL_RL_T, _ = coherent_avg(HR_bursts_LL_RL_T)
    HR_change_awake_LL_RL_T, _ = coherent_avg(HR_bursts_awake_LL_RL_T)
    HR_change_sleep_LL_RL_T, _ = coherent_avg(HR_bursts_sleep_LL_RL_T)

    #### LL, RL or LL, RL, T ####
    bursts_awake_LL_RL_or_LL_RL_T = bursts_LL_RL_or_LL_RL_T.loc[(bursts_LL_RL_or_LL_RL_T["sub_ID"] == sub) & (bursts_LL_RL_or_LL_RL_T["SIB"] == 0)].reset_index(drop=True)
    bursts_sleep_LL_RL_or_LL_RL_T = bursts_LL_RL_or_LL_RL_T.loc[(bursts_LL_RL_or_LL_RL_T["sub_ID"] == sub) & (bursts_LL_RL_or_LL_RL_T["SIB"] == 1)].reset_index(drop=True)
    # Cut according to start_sleep and end_sleep
    bursts_ll_rl_or_ll_rl_t = bursts_LL_RL_or_LL_RL_T.loc[(bursts_LL_RL_or_LL_RL_T["Start"] >= start_sleep) & (bursts_LL_RL_or_LL_RL_T["End"] <= end_sleep)].reset_index(drop=True) # I dont select the current sub, the time takes care of it already
    bursts_awake_LL_RL_or_LL_RL_T = bursts_awake_LL_RL_or_LL_RL_T.loc[(bursts_awake_LL_RL_or_LL_RL_T["Start"] >= start_sleep) & (bursts_awake_LL_RL_or_LL_RL_T["End"] <= end_sleep)]
    bursts_sleep_LL_RL_or_LL_RL_T = bursts_sleep_LL_RL_or_LL_RL_T.loc[(bursts_sleep_LL_RL_or_LL_RL_T["Start"] >= start_sleep) & (bursts_sleep_LL_RL_or_LL_RL_T["End"] <= end_sleep)]

    HR_bursts_LL_RL_or_LL_RL_T, _, _, AUC_LL_RL_or_LL_RL_T, _ = detect_HR_change_from_RR(filter_bursts(bursts_ll_rl_or_ll_rl_t), hr_df, offsets[sub]+1, plot = False)
    HR_bursts_awake_LL_RL_or_LL_RL_T, _, _, AUC_awake_LL_RL_or_LL_RL_T, _ = detect_HR_change_from_RR(filter_bursts(bursts_awake_LL_RL_or_LL_RL_T), hr_df, offsets[sub]+1, plot = False)
    HR_bursts_sleep_LL_RL_or_LL_RL_T, _, _, AUC_sleep_LL_RL_or_LL_R_T, _ = detect_HR_change_from_RR(filter_bursts(bursts_sleep_LL_RL_or_LL_RL_T), hr_df, offsets[sub]+1, plot = False)
    HR_change_LL_RL_or_LL_RL_T, _ = coherent_avg(HR_bursts_LL_RL_or_LL_RL_T)
    HR_change_awake_LL_RL_or_LL_RL_T, _ = coherent_avg(HR_bursts_awake_LL_RL_or_LL_RL_T)
    HR_change_sleep_LL_RL_or_LL_RL_T, _ = coherent_avg(HR_bursts_sleep_LL_RL_or_LL_RL_T)

    #### SINGLE LIMBS (either LL, RL, LW, RW) ####
    bursts_awake_singleLimb = bursts_single_limbs.loc[(bursts_single_limbs["sub_ID"] == sub) & (bursts_single_limbs["SIB"] == 0)].reset_index(drop=True)
    bursts_sleep_singleLimb = bursts_single_limbs.loc[(bursts_single_limbs["sub_ID"] == sub) & (bursts_single_limbs["SIB"] == 1)].reset_index(drop=True)
    # Cut according to start_sleep and end_sleep
    bursts_singleLimb = bursts_single_limbs.loc[(bursts_single_limbs["Start"] >= start_sleep) & (bursts_single_limbs["End"] <= end_sleep)].reset_index(drop=True) # I dont select the current sub, the time takes care of it already
    bursts_awake_singleLimb = bursts_awake_singleLimb.loc[(bursts_awake_singleLimb["Start"] >= start_sleep) & (bursts_awake_singleLimb["End"] <= end_sleep)]
    bursts_sleep_singleLimb = bursts_sleep_singleLimb.loc[(bursts_sleep_singleLimb["Start"] >= start_sleep) & (bursts_sleep_singleLimb["End"] <= end_sleep)]

    HR_bursts_singleLimb, HR_peak_singleLimb, latency_HR_peak_singleLimb, AUC_singleLimb, _ = detect_HR_change_from_RR(filter_bursts(bursts_singleLimb), hr_df, offsets[sub]+1, plot = False)
    HR_bursts_awake_singleLimb, _, _, AUC_awake_singleLimb, _ = detect_HR_change_from_RR(filter_bursts(bursts_awake_singleLimb), hr_df, offsets[sub]+1, plot = False)
    HR_bursts_sleep_singleLimb, _, _, AUC_sleep_singleLimb, _ = detect_HR_change_from_RR(filter_bursts(bursts_sleep_singleLimb), hr_df, offsets[sub]+1, plot = False)
    HR_change_singleLimb, _ = coherent_avg(HR_bursts_singleLimb)
    HR_change_awake_singleLimb, _ = coherent_avg(HR_bursts_awake_singleLimb)
    HR_change_sleep_singleLimb, _ = coherent_avg(HR_bursts_sleep_singleLimb)


    # Save the results
    #### 5 ALL LIMBS - low, med, high ####
    hr_responses_spt[sub] = HR_change
    hr_responses_high[sub] = HR_change_high
    hr_responses_med[sub] = HR_change_med
    hr_responses_low[sub] = HR_change_low   
    hr_responses_awake[sub] = HR_change_awake
    hr_responses_awake_low[sub] = HR_change_awake_low
    hr_responses_awake_med[sub] = HR_change_awake_med
    hr_responses_awake_high[sub] = HR_change_awake_high
    hr_responses_sleep[sub] = HR_change_sleep
    hr_responses_sleep_low[sub] = HR_change_sleep_low
    hr_responses_sleep_med[sub] = HR_change_sleep_med
    hr_responses_sleep_high[sub] = HR_change_sleep_high
    hr_peaks_spt[sub] = np.mean(HR_peak)
    hr_peaks_sleep_high[sub] = np.mean(HR_peak_sleep_high)
    hr_peaks_sleep_med[sub] = np.mean(HR_peak_sleep_med)
    hr_peaks_sleep_low[sub] = np.mean(HR_peak_sleep_low)
    
    #### LW ####
    hr_responses_LW[sub] = HR_change_lw
    hr_responses_awake_LW[sub] = HR_change_awake_lw
    hr_responses_sleep_LW[sub] = HR_change_sleep_lw

    #### RW ####
    hr_responses_RW[sub] = HR_change_rw
    hr_responses_awake_RW[sub] = HR_change_awake_rw
    hr_responses_sleep_RW[sub] = HR_change_sleep_rw

    #### LL ####
    hr_responses_LL[sub] = HR_change_ll
    hr_responses_awake_LL[sub] = HR_change_awake_ll
    hr_responses_sleep_LL[sub] = HR_change_sleep_ll

    #### RL ####
    hr_responses_RL[sub] = HR_change_rl
    hr_responses_awake_RL[sub] = HR_change_awake_rl
    hr_responses_sleep_RL[sub] = HR_change_sleep_rl

    #### BOTH WRISTS ####
    hr_responses_wrists[sub] = HR_change_wrists
    hr_responses_awake_wrists[sub] = HR_change_awake_wrists
    hr_responses_sleep_wrists[sub] = HR_change_sleep_wrists

    #### BOTH ANKLES ####
    hr_responses_ankles[sub] = HR_change_ankles
    hr_responses_awake_ankles[sub] = HR_change_awake_ankles
    hr_responses_sleep_ankles[sub] = HR_change_sleep_ankles

    #### LL, RL, T ####
    hr_responses_LL_RL_T[sub] = HR_change_LL_RL_T
    hr_responses_awake_LL_RL_T[sub] = HR_change_awake_LL_RL_T
    hr_responses_sleep_LL_RL_T[sub] = HR_change_sleep_LL_RL_T

    #### LL, RL or LL, RL, T ####
    hr_responses_LL_RL_or_LL_RL_T[sub] = HR_change_LL_RL_or_LL_RL_T
    hr_responses_awake_LL_RL_or_LL_RL_T[sub] = HR_change_awake_LL_RL_or_LL_RL_T
    hr_responses_sleep_LL_RL_or_LL_RL_T[sub] = HR_change_sleep_LL_RL_or_LL_RL_T

    #### SINGLE LIMBS ####
    hr_responses_single_limbs[sub] = HR_change_singleLimb
    hr_responses_awake_single_limbs[sub] = HR_change_awake_singleLimb
    hr_responses_sleep_single_limbs[sub] = HR_change_sleep_singleLimb
    hr_peaks_singleLimbs[sub] = np.mean(HR_peak_singleLimb)
    hr_latencies_singleLimbs[sub] = latency_HR_peak_singleLimb

158
098
633
279
906
547
971
958
815


In [18]:
# merge hr_peaks_sleep_high, hr_peaks_sleep_med, and hr_peaks_sleep_low and boxplot
hr_peaks_sleep = pd.concat([pd.Series(hr_peaks_sleep_high), pd.Series(hr_peaks_sleep_med), pd.Series(hr_peaks_sleep_low)], axis=1)
hr_peaks_sleep.columns = ["high", "med", "low"]
hr_peaks_sleep = hr_peaks_sleep.melt(var_name = "movement", value_name = "HR_peak")
hr_peaks_sleep["movement"] = hr_peaks_sleep["movement"].replace({"high": "high", "med": "med", "low": "low"})
plt.figure(figsize=(15, 9))
sns.boxplot(x = "movement", y = "HR_peak", data = hr_peaks_sleep)
plt.title("HR peaks during sleep")

Text(0.5, 1.0, 'HR peaks during sleep')

In [27]:
hr_peaks_sleep_high

{'158': 25.46374675864383,
 '098': 41.25937490757159,
 '633': 26.323995791901755,
 '279': 65.64868682335043,
 '906': 42.33088655083718,
 '547': 24.623147647583785,
 '971': 57.5624269720501,
 '958': 43.8831103216601,
 '815': 36.088537268543895}

In [24]:
list(hr_peaks_sleep_high.values())

[25.46374675864383,
 41.25937490757159,
 26.323995791901755,
 65.64868682335043,
 42.33088655083718,
 24.623147647583785,
 57.5624269720501,
 43.8831103216601,
 36.088537268543895]

In [28]:
# perform friedman test
from scipy.stats import friedmanchisquare
stat, p = friedmanchisquare(list(hr_peaks_sleep_high.values()), list(hr_peaks_sleep_med.values()), list(hr_peaks_sleep_low.values()))
print(f"HR peaks during sleep: p = {p}")
p < 0.01

# wilcoxon between high and low
from scipy.stats import wilcoxon
stat, p = wilcoxon(list(hr_peaks_sleep_high.values()), list(hr_peaks_sleep_low.values()))
print(p)

# wilcoxon between high and med
stat, p = wilcoxon(list(hr_peaks_sleep_high.values()), list(hr_peaks_sleep_med.values()))
print(p)

# wilcoxon between med and low
stat, p = wilcoxon(list(hr_peaks_sleep_med.values()), list(hr_peaks_sleep_low.values()))
print(p)

HR peaks during sleep: p = 0.004827949993831463
0.01171875
0.00390625
0.49609375


In [30]:
import matplotlib.ticker as mtick

# Plot results

## All limbs

Quntifying Peak Change and Duration - All SPT - 5 limbs

In [85]:
# Is it true that the mean is 32?
hr_matrix = pd.DataFrame([hr_responses_spt[key] for key in keys], index = keys)
hr_matrix_singleLimbs = pd.DataFrame([hr_responses_single_limbs[key] for key in keys], index = keys)
hr_matrix_sleep = pd.DataFrame([hr_responses_sleep[key] for key in keys], index = keys).T
hr_matrix_awake = pd.DataFrame([hr_responses_awake[key] for key in keys], index = keys).T
t = np.arange(-19,40)

for i in hr_matrix.index:
    plt.figure(figsize=(15, 9))
    plt.axhline(y = 0, color = "black", linestyle = "--")
    plt.plot(t, hr_matrix_singleLimbs.loc[i].values, label = i)

# hr_matrix_singleLimbs.max().std()
# hr_matrix.max().std()
# hr_matrix_sleep.max().mean()
# hr_matrix_awake.max().mean()

In [86]:
0.0117 < 0.01

False

In [80]:
a = np.array((21, 26, 35, 35, 30, 23, 27, 39, 32)) # subjects from last to first

a.mean(), a.std()

(29.77777777777778, 5.672110674711208)

In [76]:
plt.close("all")

Duration

In [ ]:
hr_matrix = pd.DataFrame([hr_responses_spt[key] for key in keys], index = keys)

hr_matrix.index = np.arange(-19,40)

hr_matrix[hr_matrix < 0]

In [37]:
hr_matrix_singleLimb

,0,1,2,3,4,5,6,7,8,9,...,49,50,51,52,53,54,55,56,57,58
158,0.944004,0.222510,-0.789283,-0.738516,-0.536509,-0.727487,-0.637980,0.502987,0.980931,0.937230,...,0.252856,0.539868,0.973097,0.009197,-0.203340,-0.673399,-1.345318,-0.900150,-0.462386,-1.036517
098,-0.776837,-0.449876,-0.153587,-0.081566,-0.188798,0.159521,0.093802,-0.217129,0.463556,0.279312,...,-1.257489,-1.551977,-1.107517,-0.447950,-0.601607,-1.071605,-0.998406,-0.758107,-0.496568,-0.800235
633,-0.472375,-0.898019,-0.769857,-0.082470,-0.477044,-0.467047,-0.376418,0.397914,0.072143,1.226708,...,3.642891,3.864350,3.770050,4.109946,3.794422,3.512093,2.956157,2.942009,2.538781,1.690478
279,-1.580936,-1.809078,-1.734401,-0.830161,-0.208808,0.104698,0.268225,0.098282,1.231743,1.322031,...,3.530767,3.504387,2.190765,2.428407,2.563257,2.455849,2.111841,0.602767,0.004912,0.950347
906,1.427949,1.478903,1.063848,0.194699,-0.444883,-0.875536,-0.757085,-0.661544,-0.060406,0.122583,...,0.414261,-0.328670,0.341297,1.193754,1.986345,2.818453,2.253100,1.289498,1.028963,0.382049
547,-0.691148,-0.452376,0.098981,0.652757,-0.032384,-0.874720,-0.951737,0.044865,0.694200,0.046291,...,0.289172,0.007007,0.476946,0.338329,1.091731,1.121417,1.072171,0.071069,1.012895,0.371962
971,-2.026858,-2.479910,0.189835,1.544096,0.337965,0.881745,1.722409,0.710682,-1.153189,-1.434845,...,1.894102,0.457363,-0.165661,2.318387,1.849997,1.294155,2.460285,2.939548,1.701743,2.993710
958,-0.029355,-1.252684,-1.270290,-0.954086,-2.680859,-0.701651,0.307955,0.344634,1.025529,1.790948,...,-0.715060,1.684569,2.938768,0.977487,-0.031291,1.148318,1.107951,0.248634,0.126377,1.775811
815,1.398697,1.694567,1.021559,-0.169277,-0.197339,-0.230042,-0.047708,-0.331324,-0.444221,-0.431952,...,3.310804,3.963644,3.938032,3.781753,3.665594,3.433789,3.201822,3.503269,3.896854,3.405872


In [41]:
hr_matrix_singleLimb = pd.DataFrame([hr_responses_single_limbs[key] for key in keys], index = keys).T
hr_matrix_singleLimb.index = np.arange(-19,40)

hr_matrix_singleLimb.max().describe()


count     9.000000
mean      4.302482
std       2.657032
min       1.890461
25%       2.818453
50%       3.717857
75%       4.109946
max      10.193519
dtype: float64

In [30]:
dur = np.array((33, 39, 28, 24, 39, 36, 39, 27, 22))
dur.mean(), dur.std()

(31.88888888888889, 6.402160129283526)

In [34]:
hr_matrix.max().describe()

count     9.000000
mean     18.737784
std      12.941826
min       4.548343
25%      10.318022
50%      18.794311
75%      19.335522
max      40.090946
dtype: float64

In [42]:
keys = list(hr_responses_awake.keys())  # Assuming all dictionaries have the same keys

hr_matrix = pd.DataFrame([hr_responses_spt[key] for key in keys], index = keys)

hr_matrix = np.array([hr_responses_spt[key] for key in keys])
time_seconds = np.arange(-19,40)  # Time axis

mean_hr_matrix = np.mean(hr_matrix, axis = 0)
sem_hr_matrix = np.std(hr_matrix, axis = 0) / np.sqrt(hr_matrix.shape[0])

f, (ax1) = plt.subplots()
f.set_figheight(6)
f.set_figwidth(11)

ax1.errorbar(time_seconds, mean_hr_matrix, yerr=sem_hr_matrix, fmt='g-', linewidth=2, label=None, ecolor='green', elinewidth=1.5, capsize=3)
ax1.plot(time_seconds, mean_hr_matrix, color = 'g', marker = 'o', linewidth=2, label = "Small movement")  # Low
ax1.axvline(x=0, color='grey', linestyle='--', linewidth=1.2)
ax1.axhline(y=0, color='grey', linestyle='--', linewidth=1.2)
ax1.annotate('Movement\nonset', xy=(0, plt.ylim()[1]-6), xytext=(-60, plt.ylim()[1]-66),
             textcoords='offset points', ha='right', va='bottom', fontsize=19,
             bbox=dict(boxstyle='round,pad=0.5', fc='gray', alpha=0.2, edgecolor='black'),
             arrowprops=dict(facecolor='black', shrink=0.05, width=2))

# plt.xticks(ticks = np.arange(-20, 40, 5), labels=np.arange(-20, 40, 5), fontsize=16)
ax1.yaxis.set_major_formatter(mtick.PercentFormatter(decimals=0))
ax1.set_xlabel('Time (seconds)', fontsize = 21)
ax1.set_ylabel('HR change', fontsize = 21)
ax1.set_label('')
ax1.xaxis.set_tick_params(labelsize=20)
ax1.yaxis.set_tick_params(labelsize=20)
ax1.yaxis.set_tick_params(labelleft=True, labelsize=20)

ax1.legend(frameon = True, fancybox = True, shadow = True, fontsize = 18, loc = "upper right")
plt.grid(True)
plt.tight_layout()

### WHOLE SPT

In [9]:
keys = list(hr_responses_awake.keys())  # Assuming all dictionaries have the same keys

# keys = list(hr_responses_med.keys())  # Assuming all dictionaries have the same keys
high_hr_matrix = np.array([hr_responses_high[key] for key in keys if ~np.isnan(hr_responses_high[key]).all()])
med_hr_matrix = np.array([hr_responses_med[key] for key in keys if ~np.isnan(hr_responses_high[key]).all()])
low_hr_matrix = np.array([hr_responses_low[key] for key in keys if ~np.isnan(hr_responses_high[key]).all()])

time_seconds = np.arange(-19,40)  # Time axis

# Calculate mean HR trends for each category
mean_high_hr = np.mean(high_hr_matrix, axis=0)
mean_med_hr = np.mean(med_hr_matrix, axis=0)
mean_low_hr = np.mean(low_hr_matrix, axis=0)
sem_high_hr = np.std(high_hr_matrix, axis=0, ddof=1) / np.sqrt(high_hr_matrix.shape[0])
sem_med_hr = np.std(med_hr_matrix, axis=0, ddof=1) / np.sqrt(med_hr_matrix.shape[0])
sem_low_hr = np.std(low_hr_matrix, axis=0, ddof=1) / np.sqrt(low_hr_matrix.shape[0])

# Create boxplots for each HR type with transparency and customized outlier properties
f, (ax1) = plt.subplots()
f.set_figheight(6)
f.set_figwidth(16)

ax1.errorbar(time_seconds, mean_low_hr, yerr=sem_low_hr, fmt='g-', linewidth=2, label=None, ecolor='green', elinewidth=1.5, capsize=3)
ax1.errorbar(time_seconds, mean_med_hr, yerr=sem_med_hr, fmt='r-', linewidth=2, label=None, ecolor='red', elinewidth=1.5, capsize=3)
ax1.errorbar(time_seconds, mean_high_hr, yerr=sem_high_hr, fmt='b-', linewidth=2, label=None, ecolor='blue', elinewidth=1.5, capsize=3)

# Add mean HR trend lines
ax1.plot(time_seconds, mean_low_hr, color = 'g', marker = 'o', linewidth=2, label = "Small movement")  # Low
ax1.plot(time_seconds, mean_med_hr, color = 'r', marker = 'o', linewidth=2, label = "Medium Movement")  # Medium
ax1.plot(time_seconds, mean_high_hr, color = 'b', marker = 'o', linewidth=2, label = "Large movement")     # High

ax1.axvline(x=0, color='grey', linestyle='--', linewidth=1.2)
ax1.axhline(y=0, color='grey', linestyle='--', linewidth=1.2)
ax1.annotate('Movement\nonset', xy=(0, plt.ylim()[1]-6), xytext=(-60, plt.ylim()[1]-66),
             textcoords='offset points', ha='right', va='bottom', fontsize=19,
             bbox=dict(boxstyle='round,pad=0.5', fc='gray', alpha=0.2, edgecolor='black'),
             arrowprops=dict(facecolor='black', shrink=0.05, width=2))

# plt.xticks(ticks = np.arange(-20, 40, 5), labels=np.arange(-20, 40, 5), fontsize=16)
ax1.yaxis.set_major_formatter(mtick.PercentFormatter(decimals=0))
ax1.set_xlabel('Time (seconds)', fontsize = 21)
ax1.set_ylabel('HR change', fontsize = 21)
ax1.set_label('')
ax1.xaxis.set_tick_params(labelsize=20)
ax1.yaxis.set_tick_params(labelsize=20)
ax1.yaxis.set_tick_params(labelleft=True, labelsize=20)

ax1.legend(frameon = True, fancybox = True, shadow = True, fontsize = 18, loc = "upper right")
plt.grid(True)
plt.tight_layout()

NameError: name 'mtick' is not defined

In [61]:
plt.savefig("/Users/marcellosicbaldi/Documents/GitHub/Movement-HR-Sleep/figures/HR_response/all_limbs_WholeSPT.png", dpi=300, bbox_inches='tight')

### Sleep vs Wake

In [45]:
keys = list(hr_responses_awake.keys())  # Assuming all dictionaries have the same keys

hr_matrix_awake = np.array([hr_responses_awake[key] for key in keys if ~np.isnan(hr_responses_awake[key]).all()])
hr_matrix_sleep = np.array([hr_responses_sleep[key] for key in keys if ~np.isnan(hr_responses_sleep[key]).all()])

time_seconds = np.arange(-19,40)  # Time axis

# Calculate mean HR trends for each category
mean_hr_awake = np.mean(hr_matrix_awake, axis=0)
mean_hr_sleep = np.mean(hr_matrix_sleep, axis=0)

sem_hr_awake = np.std(hr_matrix_awake, axis=0, ddof=1) / np.sqrt(hr_matrix_awake.shape[0])
sem_hr_sleep = np.std(hr_matrix_sleep, axis=0, ddof=1) / np.sqrt(hr_matrix_sleep.shape[0])

# Create boxplots for each HR type with transparency and customized outlier properties
f, (ax1) = plt.subplots()
f.set_figheight(6)
f.set_figwidth(12)

ax1.errorbar(time_seconds, mean_hr_awake, yerr=sem_hr_awake, fmt='r-', linewidth=2, label=None, ecolor='red', elinewidth=1.5, capsize=3)
ax1.errorbar(time_seconds, mean_hr_sleep, yerr=sem_hr_sleep, fmt='b-', linewidth=2, label=None, ecolor='blue', elinewidth=1.5, capsize=3)

# Add mean HR trend lines
ax1.plot(time_seconds, mean_hr_awake, color = 'r', marker = 'o', linewidth=2, label = "Awake")  # Awake
ax1.plot(time_seconds, mean_hr_sleep, color = 'b', marker = 'o', linewidth=2, label = "Sleep")  # Sleep

ax1.axvline(x=0, color='grey', linestyle='--', linewidth=1.2)
ax1.axhline(y=0, color='grey', linestyle='--', linewidth=1.2)
ax1.annotate('Movement\nonset', xy=(0, plt.ylim()[1]-6), xytext=(-60, plt.ylim()[1]-66),
             textcoords='offset points', ha='right', va='bottom', fontsize=19,
             bbox=dict(boxstyle='round,pad=0.5', fc='gray', alpha=0.2, edgecolor='black'),
             arrowprops=dict(facecolor='black', shrink=0.05, width=2))

# plt.xticks(ticks = np.arange(-20, 40, 5), labels=np.arange(-20, 40, 5), fontsize=16)
ax1.yaxis.set_major_formatter(mtick.PercentFormatter(decimals=0))
ax1.set_xlabel('Time (seconds)', fontsize = 21)
ax1.set_ylabel('HR change', fontsize = 21)
ax1.set_label('')
ax1.xaxis.set_tick_params(labelsize=20)
ax1.yaxis.set_tick_params(labelsize=20)

ax1.legend(frameon = True, fancybox = True, shadow = True, fontsize = 18, loc = "upper right")
plt.grid(True)
plt.tight_layout()

plt.savefig("/Users/marcellosicbaldi/Documents/GitHub/Movement-HR-Sleep/figures/HR_response/all_limbs_awake_VS_sleep.png", dpi=300, bbox_inches='tight')

### Sleep and Wake separate

In [31]:
keys = list(hr_responses_awake.keys())  # Assuming all dictionaries have the same keys

# keys = list(hr_responses_med.keys())  # Assuming all dictionaries have the same keys
high_hr_matrix_awake = np.array([hr_responses_awake_high[key] for key in keys if ~np.isnan(hr_responses_awake_high[key]).all()])
med_hr_matrix_awake = np.array([hr_responses_awake_med[key] for key in keys if ~np.isnan(hr_responses_awake_med[key]).all()])
low_hr_matrix_awake = np.array([hr_responses_awake_low[key] for key in keys if ~np.isnan(hr_responses_awake_low[key]).all()])

high_hr_matrix_sleep = np.array([hr_responses_sleep_high[key] for key in keys if ~np.isnan(hr_responses_sleep_high[key]).all()])
med_hr_matrix_sleep = np.array([hr_responses_sleep_med[key] for key in keys if ~np.isnan(hr_responses_sleep_med[key]).all()])
low_hr_matrix_sleep = np.array([hr_responses_sleep_low[key] for key in keys if ~np.isnan(hr_responses_sleep_low[key]).all()])
time_seconds = np.arange(-19,40)  # Time axis

# Calculate mean HR trends for each category
mean_high_hr_awake = np.mean(high_hr_matrix_awake, axis=0)
mean_med_hr_awake = np.mean(med_hr_matrix_awake, axis=0)
mean_low_hr_awake = np.mean(low_hr_matrix_awake, axis=0)
sem_high_hr_awake = np.std(high_hr_matrix_awake, axis=0, ddof=1) / np.sqrt(high_hr_matrix_awake.shape[0])
sem_med_hr_awake = np.std(med_hr_matrix_awake, axis=0, ddof=1) / np.sqrt(med_hr_matrix_awake.shape[0])
sem_low_hr_awake = np.std(low_hr_matrix_awake, axis=0, ddof=1) / np.sqrt(low_hr_matrix_awake.shape[0])

mean_high_hr_sleep = np.mean(high_hr_matrix_sleep, axis=0)
mean_med_hr_sleep = np.mean(med_hr_matrix_sleep, axis=0)
mean_low_hr_sleep = np.mean(low_hr_matrix_sleep, axis=0)
sem_high_hr_sleep = np.std(high_hr_matrix_sleep, axis=0, ddof=1) / np.sqrt(high_hr_matrix_sleep.shape[0])
sem_med_hr_sleep = np.std(med_hr_matrix_sleep, axis=0, ddof=1) / np.sqrt(med_hr_matrix_sleep.shape[0])
sem_low_hr_sleep = np.std(low_hr_matrix_sleep, axis=0, ddof=1) / np.sqrt(low_hr_matrix_sleep.shape[0])

# Create boxplots for each HR type with transparency and customized outlier properties
f, (ax1) = plt.subplots()
f.set_figheight(6)
f.set_figwidth(16)

ax1.errorbar(time_seconds, mean_low_hr_awake, yerr=sem_low_hr_awake, fmt='g-', linewidth=2, label=None, ecolor='green', elinewidth=1.5, capsize=3)
ax1.errorbar(time_seconds, mean_med_hr_awake, yerr=sem_med_hr_awake, fmt='r-', linewidth=2, label=None, ecolor='red', elinewidth=1.5, capsize=3)
ax1.errorbar(time_seconds, mean_high_hr_awake, yerr=sem_high_hr_awake, fmt='b-', linewidth=2, label=None, ecolor='blue', elinewidth=1.5, capsize=3)

# Add mean HR trend lines
ax1.plot(time_seconds, mean_low_hr_awake, color = 'g', marker = 'o', linewidth=2, label = "Small movement")  # Low
ax1.plot(time_seconds, mean_med_hr_awake, color = 'r', marker = 'o', linewidth=2, label = "Medium Movement")  # Medium
ax1.plot(time_seconds, mean_high_hr_awake, color = 'b', marker = 'o', linewidth=2, label = "Large movement")     # High

ax1.axvline(x=0, color='grey', linestyle='--', linewidth=1.2)
ax1.axhline(y=0, color='grey', linestyle='--', linewidth=1.2)
ax1.annotate('Movement\nonset', xy=(0, plt.ylim()[1]-6), xytext=(-60, plt.ylim()[1]-66),
             textcoords='offset points', ha='right', va='bottom', fontsize=19,
             bbox=dict(boxstyle='round,pad=0.5', fc='gray', alpha=0.2, edgecolor='black'),
             arrowprops=dict(facecolor='black', shrink=0.05, width=2))

# plt.xticks(ticks = np.arange(-20, 40, 5), labels=np.arange(-20, 40, 5), fontsize=16)
ax1.yaxis.set_major_formatter(mtick.PercentFormatter(decimals=0))
ax1.set_xlabel('Time (seconds)', fontsize = 21)
ax1.set_ylabel('HR change', fontsize = 21)
ax1.set_label('')
ax1.xaxis.set_tick_params(labelsize=20)
ax1.yaxis.set_tick_params(labelsize=20)
ax1.yaxis.set_tick_params(labelleft=True, labelsize=20)

ax1.legend(frameon = True, fancybox = True, shadow = True, fontsize = 18, loc = "upper right")
plt.grid(True)
plt.tight_layout()

# plt.savefig("/Users/marcellosicbaldi/Documents/GitHub/Movement-HR-Sleep/figures/HR_response/all_limbs_Awake.png", dpi=300, bbox_inches='tight')

f, (ax1) = plt.subplots()
f.set_figheight(6)
f.set_figwidth(16)

ax1.errorbar(time_seconds, mean_low_hr_sleep, yerr=sem_low_hr_sleep, fmt='g-', linewidth=2, label=None, ecolor='green', elinewidth=1.5, capsize=3)
ax1.errorbar(time_seconds, mean_med_hr_sleep, yerr=sem_med_hr_sleep, fmt='r-', linewidth=2, label=None, ecolor='red', elinewidth=1.5, capsize=3)
ax1.errorbar(time_seconds, mean_high_hr_sleep, yerr=sem_high_hr_sleep, fmt='b-', linewidth=2, label=None, ecolor='blue', elinewidth=1.5, capsize=3)

# Add mean HR trend lines
ax1.plot(time_seconds, mean_low_hr_sleep, color = 'g', marker = 'o', linewidth=2, label = "Small movement")  # Low
ax1.plot(time_seconds, mean_med_hr_sleep, color = 'r', marker = 'o', linewidth=2, label = "Medium Movement")  # Medium
ax1.plot(time_seconds, mean_high_hr_sleep, color = 'b', marker = 'o', linewidth=2, label = "Large movement")     # High

ax1.axvline(x=0, color='grey', linestyle='--', linewidth=1.2)
ax1.axhline(y=0, color='grey', linestyle='--', linewidth=1.2)
ax1.annotate('Movement\nonset', xy=(0, plt.ylim()[1]-6), xytext=(-60, plt.ylim()[1]-66),
             textcoords='offset points', ha='right', va='bottom', fontsize=19,
             bbox=dict(boxstyle='round,pad=0.5', fc='gray', alpha=0.2, edgecolor='black'),
             arrowprops=dict(facecolor='black', shrink=0.05, width=2))

# plt.xticks(ticks = np.arange(-20, 40, 5), labels=np.arange(-20, 40, 5), fontsize=16)
ax1.yaxis.set_major_formatter(mtick.PercentFormatter(decimals=0))
ax1.set_xlabel('Time (seconds)', fontsize = 21)
ax1.set_ylabel('HR change', fontsize = 21)
ax1.set_label('')
ax1.xaxis.set_tick_params(labelsize=20)
ax1.yaxis.set_tick_params(labelsize=20)
ax1.yaxis.set_tick_params(labelleft=True, labelsize=20)

ax1.legend(frameon = True, fancybox = True, shadow = True, fontsize = 18, loc = "upper right")
plt.grid(True)
plt.tight_layout()

# plt.savefig("/Users/marcellosicbaldi/Documents/GitHub/Movement-HR-Sleep/figures/HR_response/all_limbs_Sleep.png", dpi=300, bbox_inches='tight')

#### Statistics

In [36]:
# hr_df_high_wake = pd.DataFrame(high_hr_matrix_awake, index = keys)
hr_df_high_wake.columns = list(np.arange(-19, 40, 1))
# hr_df_med_wake = pd.DataFrame(med_hr_matrix_awake, index = keys)
# hr_df_med_wake.columns = list(np.arange(-19, 40, 1))
hr_df_low_wake = pd.DataFrame(low_hr_matrix_awake, index = keys)
hr_df_low_wake.columns = list(np.arange(-19, 40, 1))

# Save to csv
hr_df_high_wake.to_csv("/Users/marcellosicbaldi/Documents/GitHub/Movement-HR-Sleep/high_awake.csv")
hr_df_med_wake.to_csv("/Users/marcellosicbaldi/Documents/GitHub/Movement-HR-Sleep/med_awake.csv")
hr_df_low_wake.to_csv("/Users/marcellosicbaldi/Documents/GitHub/Movement-HR-Sleep/low_awake.csv")

hr_df_high_sleep = pd.DataFrame(high_hr_matrix_sleep, index = keys)
hr_df_high_sleep.columns = list(np.arange(-19, 40, 1))
hr_df_med_sleep = pd.DataFrame(med_hr_matrix_sleep, index = keys)
hr_df_med_sleep.columns = list(np.arange(-19, 40, 1))
hr_df_low_sleep = pd.DataFrame(low_hr_matrix_sleep, index = keys)
hr_df_low_sleep.columns = list(np.arange(-19, 40, 1))

# Save to csv
hr_df_high_sleep.to_csv("/Users/marcellosicbaldi/Documents/GitHub/Movement-HR-Sleep/high_sleep.csv")
hr_df_med_sleep.to_csv("/Users/marcellosicbaldi/Documents/GitHub/Movement-HR-Sleep/med_sleep.csv")
hr_df_low_sleep.to_csv("/Users/marcellosicbaldi/Documents/GitHub/Movement-HR-Sleep/low_sleep.csv")


NameError: name 'hr_df_med_wake' is not defined

In [18]:
hr_responses_sleep["low"]

array([[ 1.72420114e+00,  5.67480051e-01,  2.58769777e-01,
         7.01626844e-01,  1.09943373e+00, -6.84673805e-01,
        -1.18593867e+00,  8.03419111e-01, -4.25634897e-01,
        -2.03104285e+00,  8.96560709e-01,  5.52996423e-02,
        -7.17646801e-02,  2.59473640e-01,  9.38808431e-01,
         9.97685108e-01,  1.25754452e+00,  2.70551315e+00,
         1.43470614e+00,  2.07685847e+00,  7.83665132e-01,
         2.31830695e+00,  6.02958170e+00,  8.81636938e+00,
         1.06457537e+01,  7.78541735e+00,  6.80101415e+00,
         6.19864639e-01, -3.24535842e+00, -4.04121572e+00,
        -5.26674927e+00, -4.49265714e+00, -3.72255677e+00,
        -2.75915344e+00, -2.13291749e+00, -1.53670893e+00,
        -1.15538856e+00,  3.93570701e-01,  8.65025826e-01,
         4.84214802e-01,  4.71496325e-01, -4.45372291e-01,
         4.14563297e-02,  5.67216044e-01,  5.50002683e-01,
        -6.98271051e-02,  2.00469033e+00,  1.42889129e+00,
         2.71169964e+00,  2.44403024e+00,  3.43578789e+0

#### ANOVA

Why Use ANOVA?
ANOVA is useful because it allows you to:

Compare multiple groups at once (e.g., high, medium, and low movements) without increasing the risk of Type I errors (false positives) that would occur if you conducted multiple t-tests.
Detect interaction effects, which can provide deeper insights into how different factors influence the dependent variable.

In the case of our analysis: 

1. **Dependent Variable**: This is the variable we are measuring and trying to explain. In our case, the dependent variable is the heart rate (HR) response of the subjects.

2. **Independent Variables (Factors)**: These are the variables that we think might influence the dependent variable. In our case, we have two independent variables:

    - **Condition**: The type of movement (high, medium, or low).
    - **Time**: The time points at which the heart rate is measured.

3. **Interaction Effect**: An interaction effect occurs when the effect of one independent variable on the dependent variable depends on the level of another independent variable. In your case, the interaction between Condition and Time would mean that the way heart rate changes over time is different depending on whether the movement is high, medium, or low.

In [26]:
# save to csv


# load from csv
hr_df_high_sleep = pd.read_csv("/Users/marcellosicbaldi/Documents/GitHub/Movement-HR-Sleep/high_sleep.csv")
hr_df_med_sleep = pd.read_csv("/Users/marcellosicbaldi/Documents/GitHub/Movement-HR-Sleep/med_sleep.csv")
hr_df_low_sleep = pd.read_csv("/Users/marcellosicbaldi/Documents/GitHub/Movement-HR-Sleep/low_sleep.csv")
# I want to perform a statistical test to see if the HR change is significantly different between the different movement categories
# I will use a repeated measures ANOVA for this
# I will use the statsmodels library to perform this test
# I will use the HR change values for each subject and each movement category as the dependent variable
# The independent variable will be the movement category
# The subject will be the repeated measure 

hr_df_high_sleep["Condition"] = "High"
hr_df_med_sleep["Condition"] = "Medium"
hr_df_low_sleep["Condition"] = "Low"

# Concatenate the dataframes
combined_data = pd.concat([hr_df_high_sleep, hr_df_med_sleep, hr_df_low_sleep], ignore_index = True)

# Reshape the data to long format
long_format = pd.melt(combined_data, id_vars=['Unnamed: 0', 'Condition'], var_name='Time', value_name='HeartRate')

# Rename columns for clarity
long_format.rename(columns={'Unnamed: 0': 'Subject'}, inplace=True)

long_format.head()

,Subject,Condition,Time,HeartRate
0,158,High,-19,-0.484264
1,98,High,-19,-1.021812
2,633,High,-19,-0.854971
3,279,High,-19,-2.545230
4,906,High,-19,1.867708


The data is now combined and reshaped into a long format, making it suitable for repeated measures ANOVA. The next step is to perform the repeated measures ANOVA to compare the HR response between the three conditions.

In [28]:
from statsmodels.stats.anova import AnovaRM

# Convert Time to numeric
long_format['Time'] = long_format['Time'].astype(float)

# Perform repeated measures ANOVA
anova = AnovaRM(long_format, 'HeartRate', 'Subject', within=['Condition', 'Time'])
anova_results = anova.fit()

anova_results.summary(

/Users/marcellosicbaldi/anaconda3/lib/python3.10/site-packages/statsmodels/iolib/summary2.py:579: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  dat = dat.applymap(lambda x: _formatter(x, float_format))


,F Value,Num DF,Den DF,Pr > F
Condition,9.7568,2.0000,16.0000,0.0017
Time,15.1044,58.0000,464.0000,0.0000
Condition:Time,10.9957,116.0000,928.0000,0.0000


- **movement intensity**: The F-value is 9.7568 with a p-value of 0.0017, suggesting that there is a significant difference in heart rate responses between the high, medium, and low movement conditions.

--> heart rate responses are different for high, medium, and low movements.
- **time**: The F-value is 15.1044 with a p-value < 0.0001, indicating a significant change in heart rate over time.

--> heart rate changes over the different time points regardless of the condition

- **Interaction**: The F-value is 10.9957 with a p-value < 0.0001, indicating a significant interaction effect between condition and time on heart rate response. 

--> The way heart rate changes over time (The time course of HR) differs significantly between the conditions

SyntaxError: invalid syntax (2430763147.py, line 1)

## Single limbs alone (Only `LW`, Only `RW`, Only `LL`, Only `RL`)

### Whole SPT

In [38]:
keys = list(hr_responses_awake.keys())  # Assuming all dictionaries have the same keys

# keys = list(hr_responses_med.keys())  # Assuming all dictionaries have the same keys

hr_matrix_LW = np.array([hr_responses_LW[key] for key in keys if ~np.isnan(hr_responses_LW[key]).all()])
hr_matrix_RW = np.array([hr_responses_RW[key] for key in keys if ~np.isnan(hr_responses_RW[key]).all()])
hr_matrix_LL = np.array([hr_responses_LL[key] for key in keys if ~np.isnan(hr_responses_LL[key]).all()])
hr_matrix_RL = np.array([hr_responses_RL[key] for key in keys if ~np.isnan(hr_responses_RL[key]).all()])

time_seconds = np.arange(-19,40)  # Time axis

# Calculate mean HR trends for each category
mean_hr_LW = np.mean(hr_matrix_LW, axis=0)
mean_hr_RW = np.mean(hr_matrix_RW, axis=0)
mean_hr_LL = np.mean(hr_matrix_LL, axis=0)
mean_hr_RL = np.mean(hr_matrix_RL, axis=0)

sem_hr_LW = np.std(hr_matrix_LW, axis=0, ddof=1) / np.sqrt(hr_matrix_LW.shape[0])
sem_hr_RW = np.std(hr_matrix_RW, axis=0, ddof=1) / np.sqrt(hr_matrix_RW.shape[0])
sem_hr_LL = np.std(hr_matrix_LL, axis=0, ddof=1) / np.sqrt(hr_matrix_LL.shape[0])
sem_hr_RL = np.std(hr_matrix_RL, axis=0, ddof=1) / np.sqrt(hr_matrix_RL.shape[0])

# Create boxplots for each HR type with transparency and customized outlier properties
f, (ax1) = plt.subplots(2,2, sharex=True, sharey=True)
f.set_figheight(11)
f.set_figwidth(19)

ax1[0,0].errorbar(time_seconds, mean_hr_LW, yerr=sem_hr_LW, fmt='g-', linewidth=2, label=None, ecolor='green', elinewidth=1.5, capsize=3)
ax1[0,1].errorbar(time_seconds, mean_hr_RW, yerr=sem_hr_RW, fmt='r-', linewidth=2, label=None, ecolor='red', elinewidth=1.5, capsize=3)
ax1[1,0].errorbar(time_seconds, mean_hr_LL, yerr=sem_hr_LL, fmt='b-', linewidth=2, label=None, ecolor='blue', elinewidth=1.5, capsize=3)
ax1[1,1].errorbar(time_seconds, mean_hr_RL, yerr=sem_hr_RL, fmt='m-', linewidth=2, label=None, ecolor='magenta', elinewidth=1.5, capsize=3)

# Add mean HR trend lines
ax1[0,0].plot(time_seconds, mean_hr_LW, color = 'g', marker = 'o', linewidth=2, label = "Left wrist")  # Left wrist
ax1[0,1].plot(time_seconds, mean_hr_RW, color = 'r', marker = 'o', linewidth=2, label = "Right wrist")  # Right wrist
ax1[1,0].plot(time_seconds, mean_hr_LL, color = 'b', marker = 'o', linewidth=2, label = "Left ankle")  # Left ankle
ax1[1,1].plot(time_seconds, mean_hr_RL, color = 'm', marker = 'o', linewidth=2, label = "Right ankle")  # Right ankle

for i in range(2):
    for j in range(2):
        ax1[i,j].axvline(x=0, color='grey', linestyle='--', linewidth=1.2)
        ax1[i,j].axhline(y=0, color='grey', linestyle='--', linewidth=1.2)
        ax1[i,j].annotate('Movement\nonset', xy=(0, plt.ylim()[1]-6), xytext=(-60, plt.ylim()[1]),
                 textcoords='offset points', ha='right', va='bottom', fontsize=19,
                 bbox=dict(boxstyle='round,pad=0.5', fc='gray', alpha=0.2, edgecolor='black'),
                 arrowprops=dict(facecolor='black', shrink=0.05, width=2))

        ax1[i,j].yaxis.set_major_formatter(mtick.PercentFormatter(decimals=0))
        ax1[i,j].set_xlabel('Time (seconds)', fontsize = 21)
        ax1[i,j].set_ylabel('HR change', fontsize = 21)
        ax1[i,j].set_label('')
        ax1[i,j].xaxis.set_tick_params(labelsize=20)
        ax1[i,j].yaxis.set_tick_params(labelsize=20)
        ax1[i,j].yaxis.set_tick_params(labelleft=True, labelsize=20)

        ax1[i,j].legend(frameon = True, fancybox = True, shadow = True, fontsize = 18, loc = "upper right")
        ax1[i,j].grid(True)
        plt.tight_layout()

# plt.savefig("/Users/marcellosicbaldi/Documents/GitHub/Movement-HR-Sleep/figures/HR_response/single_limbs_LW_RW_LL_RL_WholeSPT.png", dpi=300, bbox_inches='tight')

### Sleep vs Wake

In [39]:
keys = list(hr_responses_awake.keys())  # Assuming all dictionaries have the same keys

# keys = list(hr_responses_med.keys())  # Assuming all dictionaries have the same keys

hr_matrix_LW_sleep = np.array([hr_responses_sleep_LW[key] for key in keys if ~np.isnan(hr_responses_sleep_LW[key]).all()])
hr_matrix_RW_sleep = np.array([hr_responses_sleep_RW[key] for key in keys if ~np.isnan(hr_responses_sleep_RW[key]).all()])
hr_matrix_LL_sleep = np.array([hr_responses_sleep_LL[key] for key in keys if ~np.isnan(hr_responses_sleep_LL[key]).all()])
hr_matrix_RL_sleep = np.array([hr_responses_sleep_RL[key] for key in keys if ~np.isnan(hr_responses_sleep_RL[key]).all()])

hr_matrix_LW_awake = np.array([hr_responses_awake_LW[key] for key in keys if ~np.isnan(hr_responses_awake_LW[key]).all()])
hr_matrix_RW_awake = np.array([hr_responses_awake_RW[key] for key in keys if ~np.isnan(hr_responses_awake_RW[key]).all()])
hr_matrix_LL_awake = np.array([hr_responses_awake_LL[key] for key in keys if ~np.isnan(hr_responses_awake_LL[key]).all()])
hr_matrix_RL_awake = np.array([hr_responses_awake_RL[key] for key in keys if ~np.isnan(hr_responses_awake_RL[key]).all()])

time_seconds = np.arange(-19,40)  # Time axis

# Calculate mean HR trends for each category
mean_hr_LW_sleep = np.mean(hr_matrix_LW_sleep, axis=0)
mean_hr_RW_sleep = np.mean(hr_matrix_RW_sleep, axis=0)
mean_hr_LL_sleep = np.mean(hr_matrix_LL_sleep, axis=0)
mean_hr_RL_sleep = np.mean(hr_matrix_RL_sleep, axis=0)

sem_hr_LW_sleep = np.std(hr_matrix_LW_sleep, axis=0, ddof=1) / np.sqrt(hr_matrix_LW_sleep.shape[0])
sem_hr_RW_sleep = np.std(hr_matrix_RW_sleep, axis=0, ddof=1) / np.sqrt(hr_matrix_RW_sleep.shape[0])
sem_hr_LL_sleep = np.std(hr_matrix_LL_sleep, axis=0, ddof=1) / np.sqrt(hr_matrix_LL_sleep.shape[0])
sem_hr_RL_sleep = np.std(hr_matrix_RL_sleep, axis=0, ddof=1) / np.sqrt(hr_matrix_RL_sleep.shape[0])

mean_hr_LW_awake = np.mean(hr_matrix_LW_awake, axis=0)
mean_hr_RW_awake = np.mean(hr_matrix_RW_awake, axis=0)
mean_hr_LL_awake = np.mean(hr_matrix_LL_awake, axis=0)
mean_hr_RL_awake = np.mean(hr_matrix_RL_awake, axis=0)

sem_hr_LW_awake = np.std(hr_matrix_LW_awake, axis=0, ddof=1) / np.sqrt(hr_matrix_LW_awake.shape[0])
sem_hr_RW_awake = np.std(hr_matrix_RW_awake, axis=0, ddof=1) / np.sqrt(hr_matrix_RW_awake.shape[0])
sem_hr_LL_awake = np.std(hr_matrix_LL_awake, axis=0, ddof=1) / np.sqrt(hr_matrix_LL_awake.shape[0])
sem_hr_RL_awake = np.std(hr_matrix_RL_awake, axis=0, ddof=1) / np.sqrt(hr_matrix_RL_awake.shape[0])

# Create boxplots for each HR type with transparency and customized outlier properties
f, (ax1) = plt.subplots(2,2, sharex=True, sharey=True)
f.set_figheight(11)
f.set_figwidth(19)

ax1[0,0].errorbar(time_seconds, mean_hr_LW_awake, yerr=sem_hr_LW_awake, fmt='g-', linewidth=2, label=None, ecolor='green', elinewidth=1.5, capsize=3)
ax1[0,1].errorbar(time_seconds, mean_hr_RW_awake, yerr=sem_hr_RW_awake, fmt='r-', linewidth=2, label=None, ecolor='red', elinewidth=1.5, capsize=3)
ax1[1,0].errorbar(time_seconds, mean_hr_LL_awake, yerr=sem_hr_LL_awake, fmt='b-', linewidth=2, label=None, ecolor='blue', elinewidth=1.5, capsize=3)
ax1[1,1].errorbar(time_seconds, mean_hr_RL_awake, yerr=sem_hr_RL_awake, fmt='m-', linewidth=2, label=None, ecolor='magenta', elinewidth=1.5, capsize=3)

# Add mean HR trend lines
ax1[0,0].plot(time_seconds, mean_hr_LW_awake, color = 'g', marker = 'o', linewidth=2, label = "Left wrist")  # Left wrist
ax1[0,1].plot(time_seconds, mean_hr_RW_awake, color = 'r', marker = 'o', linewidth=2, label = "Right wrist")  # Right wrist
ax1[1,0].plot(time_seconds, mean_hr_LL_awake, color = 'b', marker = 'o', linewidth=2, label = "Left ankle")  # Left ankle
ax1[1,1].plot(time_seconds, mean_hr_RL_awake, color = 'm', marker = 'o', linewidth=2, label = "Right ankle")  # Right ankle

for i in range(2):
    for j in range(2):
        ax1[i,j].axvline(x=0, color='grey', linestyle='--', linewidth=1.2)
        ax1[i,j].axhline(y=0, color='grey', linestyle='--', linewidth=1.2)
        ax1[i,j].annotate('Movement\nonset', xy=(0, plt.ylim()[1]-6), xytext=(-60, plt.ylim()[1]),
                 textcoords='offset points', ha='right', va='bottom', fontsize=19,
                 bbox=dict(boxstyle='round,pad=0.5', fc='gray', alpha=0.2, edgecolor='black'),
                 arrowprops=dict(facecolor='black', shrink=0.05, width=2))

        ax1[i,j].yaxis.set_major_formatter(mtick.PercentFormatter(decimals=0))
        ax1[i,j].set_xlabel('Time (seconds)', fontsize = 21)
        ax1[i,j].set_ylabel('HR change', fontsize = 21)
        ax1[i,j].set_label('')
        ax1[i,j].xaxis.set_tick_params(labelsize=20)
        ax1[i,j].yaxis.set_tick_params(labelsize=20)
        ax1[i,j].yaxis.set_tick_params(labelleft=True, labelsize=20)

        ax1[i,j].legend(frameon = True, fancybox = True, shadow = True, fontsize = 18, loc = "upper right")
        ax1[i,j].grid(True)
        plt.tight_layout()

plt.savefig("/Users/marcellosicbaldi/Documents/GitHub/Movement-HR-Sleep/figures/HR_response/single_limbs_LW_RW_LL_RL_Awake.png", dpi=300, bbox_inches='tight')

f, (ax1) = plt.subplots(2,2, sharex=True, sharey=True)
f.set_figheight(11)
f.set_figwidth(19)

ax1[0,0].errorbar(time_seconds, mean_hr_LW_sleep, yerr=sem_hr_LW_sleep, fmt='g-', linewidth=2, label=None, ecolor='green', elinewidth=1.5, capsize=3)
ax1[0,1].errorbar(time_seconds, mean_hr_RW_sleep, yerr=sem_hr_RW_sleep, fmt='r-', linewidth=2, label=None, ecolor='red', elinewidth=1.5, capsize=3)
ax1[1,0].errorbar(time_seconds, mean_hr_LL_sleep, yerr=sem_hr_LL_sleep, fmt='b-', linewidth=2, label=None, ecolor='blue', elinewidth=1.5, capsize=3)
ax1[1,1].errorbar(time_seconds, mean_hr_RL_sleep, yerr=sem_hr_RL_sleep, fmt='m-', linewidth=2, label=None, ecolor='magenta', elinewidth=1.5, capsize=3)

# Add mean HR trend lines
ax1[0,0].plot(time_seconds, mean_hr_LW_sleep, color = 'g', marker = 'o', linewidth=2, label = "Left wrist")  # Left wrist
ax1[0,1].plot(time_seconds, mean_hr_RW_sleep, color = 'r', marker = 'o', linewidth=2, label = "Right wrist")  # Right wrist
ax1[1,0].plot(time_seconds, mean_hr_LL_sleep, color = 'b', marker = 'o', linewidth=2, label = "Left ankle")  # Left ankle
ax1[1,1].plot(time_seconds, mean_hr_RL_sleep, color = 'm', marker = 'o', linewidth=2, label = "Right ankle")  # Right ankle

for i in range(2):
    for j in range(2):
        ax1[i,j].axvline(x=0, color='grey', linestyle='--', linewidth=1.2)
        ax1[i,j].axhline(y=0, color='grey', linestyle='--', linewidth=1.2)
        ax1[i,j].annotate('Movement\nonset', xy=(0, plt.ylim()[1]-6), xytext=(-60, plt.ylim()[1]),
                 textcoords='offset points', ha='right', va='bottom', fontsize=19,
                 bbox=dict(boxstyle='round,pad=0.5', fc='gray', alpha=0.2, edgecolor='black'),
                 arrowprops=dict(facecolor='black', shrink=0.05, width=2))

        ax1[i,j].yaxis.set_major_formatter(mtick.PercentFormatter(decimals=0))
        ax1[i,j].set_xlabel('Time (seconds)', fontsize = 21)
        ax1[i,j].set_ylabel('HR change', fontsize = 21)
        ax1[i,j].set_label('')
        ax1[i,j].xaxis.set_tick_params(labelsize=20)
        ax1[i,j].yaxis.set_tick_params(labelsize=20)
        ax1[i,j].yaxis.set_tick_params(labelleft=True, labelsize=20)

        ax1[i,j].legend(frameon = True, fancybox = True, shadow = True, fontsize = 18, loc = "upper right")
        ax1[i,j].grid(True)
        plt.tight_layout()

# plt.savefig("/Users/marcellosicbaldi/Documents/GitHub/Movement-HR-Sleep/figures/HR_response/single_limbs_LW_RW_LL_RL_Sleep.png", dpi=300, bbox_inches='tight')

/Users/marcellosicbaldi/anaconda3/lib/python3.10/site-packages/numpy/core/_methods.py:269: RuntimeWarning: Degrees of freedom <= 0 for slice
  ret = _var(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
/Users/marcellosicbaldi/anaconda3/lib/python3.10/site-packages/numpy/core/_methods.py:258: RuntimeWarning: invalid value encountered in divide
  ret = um.true_divide(


## Single Limbs (Either `LW`, `RW`, `LL`, `RL`)

In [35]:
keys = list(hr_responses_awake.keys())  # Assuming all dictionaries have the same keys

# keys = list(hr_responses_med.keys())  # Assuming all dictionaries have the same keys

hr_matrix_singleLimb = np.array([hr_responses_single_limbs[key] for key in keys if ~np.isnan(hr_responses_single_limbs[key]).all()])
hr_matrix_singleLimb_awake = np.array([hr_responses_awake_single_limbs[key] for key in keys if ~np.isnan(hr_responses_awake_single_limbs[key]).all()])
hr_matrix_singleLimb_sleep = np.array([hr_responses_sleep_single_limbs[key] for key in keys if ~np.isnan(hr_responses_sleep_single_limbs[key]).all()])

time_seconds = np.arange(-19,40)  # Time axis

# Calculate mean HR trends for each category
mean_hr_singleLimb = np.mean(hr_matrix_singleLimb, axis=0)
mean_hr_singleLimb_awake = np.mean(hr_matrix_singleLimb_awake, axis=0)
mean_hr_singleLimb_sleep = np.mean(hr_matrix_singleLimb_sleep, axis=0)

sem_hr_singleLimb = np.std(hr_matrix_singleLimb, axis=0, ddof=1) / np.sqrt(hr_matrix_singleLimb.shape[0])
sem_hr_singleLimb_awake = np.std(hr_matrix_singleLimb_awake, axis=0, ddof=1) / np.sqrt(hr_matrix_singleLimb_awake.shape[0])
sem_hr_singleLimb_sleep = np.std(hr_matrix_singleLimb_sleep, axis=0, ddof=1) / np.sqrt(hr_matrix_singleLimb_sleep.shape[0])

# Create boxplots for each HR type with transparency and customized outlier properties
f, (ax1) = plt.subplots(1,3, sharex=True, sharey=True)
f.set_figheight(6)
f.set_figwidth(19)

ax1[0].errorbar(time_seconds, mean_hr_singleLimb, yerr=sem_hr_singleLimb, fmt='g-', linewidth=2, label=None, ecolor='green', elinewidth=1.5, capsize=3)
ax1[1].errorbar(time_seconds, mean_hr_singleLimb_awake, yerr=sem_hr_singleLimb_awake, fmt='r-', linewidth=2, label=None, ecolor='red', elinewidth=1.5, capsize=3)
ax1[2].errorbar(time_seconds, mean_hr_singleLimb_sleep, yerr=sem_hr_singleLimb_sleep, fmt='b-', linewidth=2, label=None, ecolor='blue', elinewidth=1.5, capsize=3)

# Add mean HR trend lines
ax1[0].plot(time_seconds, mean_hr_singleLimb, color = 'g', marker = 'o', linewidth=2, label = "Single limb")  # Single limb
ax1[1].plot(time_seconds, mean_hr_singleLimb_awake, color = 'r', marker = 'o', linewidth=2, label = "Single limb")  # Single limb
ax1[2].plot(time_seconds, mean_hr_singleLimb_sleep, color = 'b', marker = 'o', linewidth=2, label = "Single limb")  # Single limb

for i in range(3):
    ax1[i].axvline(x=0, color='grey', linestyle='--', linewidth=1.2)
    ax1[i].axhline(y=0, color='grey', linestyle='--', linewidth=1.2)
    ax1[i].annotate('Movement\nonset', xy=(0, plt.ylim()[1]-6), xytext=(-60, plt.ylim()[1]),
             textcoords='offset points', ha='right', va='bottom', fontsize=19,
             bbox=dict(boxstyle='round,pad=0.5', fc='gray', alpha=0.2, edgecolor='black'),
             arrowprops=dict(facecolor='black', shrink=0.05, width=2))

    ax1[i].yaxis.set_major_formatter(mtick.PercentFormatter(decimals=0))
    ax1[i].set_xlabel('Time (seconds)', fontsize = 21)
    ax1[i].set_ylabel('HR change', fontsize = 21)
    ax1[i].set_label('')
    ax1[i].xaxis.set_tick_params(labelsize=20)
    ax1[i].yaxis.set_tick_params(labelsize=20)
    ax1[i].yaxis.set_tick_params(labelleft=True, labelsize=20)

    ax1[i].legend(frameon = True, fancybox = True, shadow = True, fontsize = 18, loc = "upper right")
    ax1[i].grid(True)
    ax1[i].set_title(['Whole SPT', 'Awake', 'Sleep'][i], fontsize = 21)
    plt.tight_layout()

plt.savefig("/Users/marcellosicbaldi/Documents/GitHub/Movement-HR-Sleep/figures/HR_response/single_limbs.png", dpi=300, bbox_inches='tight')

## LL, RL, T

In [36]:
keys = list(hr_responses_awake.keys())  # Assuming all dictionaries have the same keys

# keys = list(hr_responses_med.keys())  # Assuming all dictionaries have the same keys

hr_matrix_LL_RL_T = np.array([hr_responses_LL_RL_T[key] for key in keys if ~np.isnan(hr_responses_LL_RL_T[key]).all()])
hr_matrix_LL_RL_T_awake = np.array([hr_responses_awake_LL_RL_T[key] for key in keys if ~np.isnan(hr_responses_awake_LL_RL_T[key]).all()])
hr_matrix_LL_RL_T_sleep = np.array([hr_responses_sleep_LL_RL_T[key] for key in keys if ~np.isnan(hr_responses_sleep_LL_RL_T[key]).all()])

time_seconds = np.arange(-19,40)  # Time axis

# Calculate mean HR trends for each category
mean_hr_LL_RL_T = np.mean(hr_matrix_LL_RL_T, axis=0)
mean_hr_LL_RL_T_awake = np.mean(hr_matrix_LL_RL_T_awake, axis=0)
mean_hr_LL_RL_T_sleep = np.mean(hr_matrix_LL_RL_T_sleep, axis=0)

sem_hr_LL_RL_T = np.std(hr_matrix_LL_RL_T, axis=0, ddof=1) / np.sqrt(hr_matrix_LL_RL_T.shape[0])
sem_hr_LL_RL_T_awake = np.std(hr_matrix_LL_RL_T_awake, axis=0, ddof=1) / np.sqrt(hr_matrix_LL_RL_T_awake.shape[0])
sem_hr_LL_RL_T_sleep = np.std(hr_matrix_LL_RL_T_sleep, axis=0, ddof=1) / np.sqrt(hr_matrix_LL_RL_T_sleep.shape[0])

# Create boxplots for each HR type with transparency and customized outlier properties
f, (ax1) = plt.subplots(1,3, sharex=True, sharey=True)
f.set_figheight(6)
f.set_figwidth(19)

ax1[0].errorbar(time_seconds, mean_hr_LL_RL_T, yerr=sem_hr_LL_RL_T, fmt='g-', linewidth=2, label=None, ecolor='green', elinewidth=1.5, capsize=3)
ax1[1].errorbar(time_seconds, mean_hr_LL_RL_T_awake, yerr=sem_hr_LL_RL_T_awake, fmt='r-', linewidth=2, label=None, ecolor='red', elinewidth=1.5, capsize=3)
ax1[2].errorbar(time_seconds, mean_hr_LL_RL_T_sleep, yerr=sem_hr_LL_RL_T_sleep, fmt='b-', linewidth=2, label=None, ecolor='blue', elinewidth=1.5, capsize=3)

# Add mean HR trend lines
ax1[0].plot(time_seconds, mean_hr_LL_RL_T, color = 'g', marker = 'o', linewidth=2, label = "LL_RL_T")  # LL_RL_T
ax1[1].plot(time_seconds, mean_hr_LL_RL_T_awake, color = 'r', marker = 'o', linewidth=2, label = "LL_RL_T")  # LL_RL_T
ax1[2].plot(time_seconds, mean_hr_LL_RL_T_sleep, color = 'b', marker = 'o', linewidth=2, label = "LL_RL_T")  # LL_RL_T

for i in range(3):
    ax1[i].axvline(x=0, color='grey', linestyle='--', linewidth=1.2)
    ax1[i].axhline(y=0, color='grey', linestyle='--', linewidth=1.2)
    ax1[i].annotate('Movement\nonset', xy=(0, plt.ylim()[1]-6), xytext=(-60, plt.ylim()[1]),
             textcoords='offset points', ha='right', va='bottom', fontsize=19,
             bbox=dict(boxstyle='round,pad=0.5', fc='gray', alpha=0.2, edgecolor='black'),
             arrowprops=dict(facecolor='black', shrink=0.05, width=2))

    ax1[i].yaxis.set_major_formatter(mtick.PercentFormatter(decimals=0))
    ax1[i].set_xlabel('Time (seconds)', fontsize = 21)
    ax1[i].set_ylabel('HR change', fontsize = 21)
    ax1[i].set_label('')
    ax1[i].xaxis.set_tick_params(labelsize=20)
    ax1[i].yaxis.set_tick_params(labelsize=20)
    ax1[i].yaxis.set_tick_params(labelleft=True, labelsize=20)

    ax1[i].legend(frameon = True, fancybox = True, shadow = True, fontsize = 18, loc = "upper right")
    ax1[i].grid(True)
    ax1[i].set_title(['Whole SPT', 'Awake', 'Sleep'][i], fontsize = 21)
    plt.tight_layout()

plt.savefig("/Users/marcellosicbaldi/Documents/GitHub/Movement-HR-Sleep/figures/HR_response/LL_RL_T.png", dpi=300, bbox_inches='tight')

/Users/marcellosicbaldi/anaconda3/lib/python3.10/site-packages/numpy/core/_methods.py:269: RuntimeWarning: Degrees of freedom <= 0 for slice
  ret = _var(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
/Users/marcellosicbaldi/anaconda3/lib/python3.10/site-packages/numpy/core/_methods.py:258: RuntimeWarning: invalid value encountered in divide
  ret = um.true_divide(


## LL. RL, T or LL, RL

In [37]:
keys = list(hr_responses_awake.keys())  # Assuming all dictionaries have the same keys

# keys = list(hr_responses_med.keys())  # Assuming all dictionaries have the same keys

hr_matrix_LL_RL_or_LL_RL_T = np.array([hr_responses_LL_RL_or_LL_RL_T[key] for key in keys if ~np.isnan(hr_responses_LL_RL_or_LL_RL_T[key]).all()])
hr_matrix_LL_RL_or_LL_RL_T_awake = np.array([hr_responses_awake_LL_RL_or_LL_RL_T[key] for key in keys if ~np.isnan(hr_responses_awake_LL_RL_or_LL_RL_T[key]).all()])
hr_matrix_LL_RL_or_LL_RL_T_sleep = np.array([hr_responses_sleep_LL_RL_or_LL_RL_T[key] for key in keys if ~np.isnan(hr_responses_sleep_LL_RL_or_LL_RL_T[key]).all()])

time_seconds = np.arange(-19,40)  # Time axis

# Calculate mean HR trends for each category
mean_hr_LL_RL_or_LL_RL_T = np.mean(hr_matrix_LL_RL_or_LL_RL_T, axis=0)
mean_hr_LL_RL_or_LL_RL_T_awake = np.mean(hr_matrix_LL_RL_or_LL_RL_T_awake, axis=0)
mean_hr_LL_RL_or_LL_RL_T_sleep = np.mean(hr_matrix_LL_RL_or_LL_RL_T_sleep, axis=0)

sem_hr_LL_RL_or_LL_RL_T = np.std(hr_matrix_LL_RL_or_LL_RL_T, axis=0, ddof=1) / np.sqrt(hr_matrix_LL_RL_or_LL_RL_T.shape[0])
sem_hr_LL_RL_or_LL_RL_T_awake = np.std(hr_matrix_LL_RL_or_LL_RL_T_awake, axis=0, ddof=1) / np.sqrt(hr_matrix_LL_RL_or_LL_RL_T_awake.shape[0])
sem_hr_LL_RL_or_LL_RL_T_sleep = np.std(hr_matrix_LL_RL_or_LL_RL_T_sleep, axis=0, ddof=1) / np.sqrt(hr_matrix_LL_RL_or_LL_RL_T_sleep.shape[0])

# Create boxplots for each HR type with transparency and customized outlier properties
f, (ax1) = plt.subplots(1,3, sharex=True, sharey=True)
f.set_figheight(6)
f.set_figwidth(19)

ax1[0].errorbar(time_seconds, mean_hr_LL_RL_or_LL_RL_T, yerr=sem_hr_LL_RL_or_LL_RL_T, fmt='g-', linewidth=2, label=None, ecolor='green', elinewidth=1.5, capsize=3)
ax1[1].errorbar(time_seconds, mean_hr_LL_RL_or_LL_RL_T_awake, yerr=sem_hr_LL_RL_or_LL_RL_T_awake, fmt='r-', linewidth=2, label=None, ecolor='red', elinewidth=1.5, capsize=3)
ax1[2].errorbar(time_seconds, mean_hr_LL_RL_or_LL_RL_T_sleep, yerr=sem_hr_LL_RL_or_LL_RL_T_sleep, fmt='b-', linewidth=2, label=None, ecolor='blue', elinewidth=1.5, capsize=3)

# Add mean HR trend lines
ax1[0].plot(time_seconds, mean_hr_LL_RL_or_LL_RL_T, color = 'g', marker = 'o', linewidth=2, label = "LL_RL_or_LL_RL_T")  # LL_RL_or_LL_RL_T
ax1[1].plot(time_seconds, mean_hr_LL_RL_or_LL_RL_T_awake, color = 'r', marker = 'o', linewidth=2, label = "LL_RL_or_LL_RL_T")  # LL_RL_or_LL_RL_T
ax1[2].plot(time_seconds, mean_hr_LL_RL_or_LL_RL_T_sleep, color = 'b', marker = 'o', linewidth=2, label = "LL_RL_or_LL_RL_T")  # LL_RL_or_LL_RL_T

for i in range(3):
    ax1[i].axvline(x=0, color='grey', linestyle='--', linewidth=1.2)
    ax1[i].axhline(y=0, color='grey', linestyle='--', linewidth=1.2)
    ax1[i].annotate('Movement\nonset', xy=(0, plt.ylim()[1]-6), xytext=(-60, plt.ylim()[1]),
             textcoords='offset points', ha='right', va='bottom', fontsize=19,
             bbox=dict(boxstyle='round,pad=0.5', fc='gray', alpha=0.2, edgecolor='black'),
             arrowprops=dict(facecolor='black', shrink=0.05, width=2))

    ax1[i].yaxis.set_major_formatter(mtick.PercentFormatter(decimals=0))
    ax1[i].set_xlabel('Time (seconds)', fontsize = 21)
    ax1[i].set_ylabel('HR change', fontsize = 21)
    ax1[i].set_label('')
    ax1[i].xaxis.set_tick_params(labelsize=20)
    ax1[i].yaxis.set_tick_params(labelsize=20)
    ax1[i].yaxis.set_tick_params(labelleft=True, labelsize=20)

    # ax1[i].legend("movement"frameon = True, fancybox = True, shadow = True, fontsize = 18, loc = "upper right")
    ax1[i].grid(True)
    ax1[i].set_title(['Whole SPT', 'Awake', 'Sleep'][i], fontsize = 21)
    plt.tight_layout()

plt.savefig("/Users/marcellosicbaldi/Documents/GitHub/Movement-HR-Sleep/figures/HR_response/LL_RL_T_or_LL_R_T.png", dpi=300, bbox_inches='tight')